In [17]:
!pip3 install catboost
!pip3 install optuna

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [ ]:

import pandas as pd
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.base import BaseEstimator, ClassifierMixin
import xgboost as xgb
import catboost as cb
import time
from sklearn.metrics import roc_auc_score

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

In [57]:
print("It's running")

It's running


In [14]:
train_data = pd.read_csv('/Users/nikitaskazutin/ML_lab1/data/train.csv')
test_data = pd.read_csv('/Users/nikitaskazutin/ML_lab1/data/test.csv')

In [16]:
def prepare_spaceship_data(train_data, test_data, val_size=0.2, random_state=42):
    print("Loading and preparing Spaceship Titanic data with preserved categorical features...")

    print(f"Training data: {train_data.shape[0]} rows, {train_data.shape[1]} columns")
    print(f"Test data: {test_data.shape[0]} rows, {test_data.shape[1]} columns")

    train_ids = train_data['PassengerId'].copy() if 'PassengerId' in train_data.columns else None
    test_ids = test_data['PassengerId'].copy() if 'PassengerId' in test_data.columns else None

    if 'Transported' in train_data.columns:
        y = train_data['Transported'].copy()
        if y.dtype == bool:
            y = y.astype(int)
            print("Converted boolean target to integer (0/1)")
        X = train_data.drop('Transported', axis=1)
    else:
        raise ValueError("No 'Transported' column found in training data")

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=val_size, random_state=random_state, stratify=y
    )

    print(f"Split into {X_train.shape[0]} training samples and {X_val.shape[0]} validation samples")

    dfs = [
        (X_train, 'train'),
        (X_val, 'val'),
        (test_data, 'test')
    ]

    combined = pd.concat([df.assign(source=source) for df, source in dfs], axis=0)

    if 'PassengerId' in combined.columns:
        combined['PassengerGroup'] = combined['PassengerId'].str.split('_').str[0].astype(int)
        combined['PassengerNumber'] = combined['PassengerId'].str.split('_').str[1].astype(int)

        group_sizes = combined['PassengerGroup'].value_counts()
        combined['GroupSize'] = combined['PassengerGroup'].map(group_sizes)
        combined['TravelingAlone'] = (combined['GroupSize'] == 1).astype(int)

    if 'Cabin' in combined.columns:
        combined['CabinDeck'] = 'Unknown'
        combined['CabinNum'] = np.nan
        combined['CabinSide'] = 'Unknown'
        combined['HasCabin'] = combined['Cabin'].notna().astype(int)

        cabin_mask = combined['Cabin'].notna()
        if cabin_mask.any():
            cabin_parts = combined.loc[cabin_mask, 'Cabin'].str.split('/', expand=True)
            if cabin_parts.shape[1] >= 3:
                combined.loc[cabin_mask, 'CabinDeck'] = cabin_parts[0]
                combined.loc[cabin_mask, 'CabinNum'] = pd.to_numeric(cabin_parts[1], errors='coerce')
                combined.loc[cabin_mask, 'CabinSide'] = cabin_parts[2]

    if 'Name' in combined.columns:
        combined['LastName'] = combined['Name'].str.split(' ').str[0]
        combined['FirstName'] = combined['Name'].str.split(' ').str[1:].str.join(' ')

        family_sizes = combined['LastName'].value_counts()
        combined['FamilySize'] = combined['LastName'].map(family_sizes)
        combined['HasFamily'] = (combined['FamilySize'] > 1).astype(int)

    num_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in num_cols:
        if col in combined.columns:
            group_medians = combined.groupby('PassengerGroup')[col].transform('median')
            combined[col] = combined[col].fillna(group_medians)

            if 'HomePlanet' in combined.columns:
                planet_medians = combined.groupby('HomePlanet')[col].transform('median')
                combined[col] = combined[col].fillna(planet_medians)

            combined[col] = combined[col].fillna(combined[col].median())

    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    spending_cols = [col for col in spending_cols if col in combined.columns]
    if spending_cols:
        combined['TotalSpend'] = combined[spending_cols].sum(axis=1)
        combined['HasSpent'] = (combined['TotalSpend'] > 0).astype(int)

        combined['TotalSpendLog'] = np.log1p(combined['TotalSpend'])
        for col in spending_cols:
            combined[f'{col}Log'] = np.log1p(combined[col])

        for col in spending_cols:
            combined[f'{col}Ratio'] = 0
            spend_mask = combined['TotalSpend'] > 0
            if spend_mask.any():
                combined.loc[spend_mask, f'{col}Ratio'] = combined.loc[spend_mask, col] / combined.loc[
                    spend_mask, 'TotalSpend']

        for col in spending_cols:
            combined[f'Used{col}'] = (combined[col] > 0).astype(int)

    cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
    cat_cols = [col for col in cat_cols if col in combined.columns]

    for col in cat_cols:
        for group in combined[combined[col].isna()]['PassengerGroup'].unique():
            group_mode = combined[(combined['PassengerGroup'] == group) & combined[col].notna()][col].mode()
            if not group_mode.empty:
                combined.loc[(combined['PassengerGroup'] == group) & combined[col].isna(), col] = group_mode.iloc[0]

        combined[col] = combined[col].fillna(combined[col].mode().iloc[0])

    if 'Age' in combined.columns:
        combined['AgeGroup'] = pd.cut(
            combined['Age'],
            bins=[0, 12, 18, 25, 40, 60, 1000],
            labels=['Child', 'Teen', 'YoungAdult', 'Adult', 'MiddleAge', 'Senior']
        )

    if 'CryoSleep' in combined.columns and 'TotalSpend' in combined.columns:
        combined['ActiveSpender'] = ((combined['CryoSleep'] == False) & (combined['TotalSpend'] > 0)).astype(int)
        combined['CryoSleepSpender'] = ((combined['CryoSleep'] == True) & (combined['TotalSpend'] > 0)).astype(int)

    if 'HomePlanet' in combined.columns and 'Destination' in combined.columns:
        combined['Route'] = combined['HomePlanet'].astype(str) + "_to_" + combined['Destination'].astype(str)

    important_cat_cols = [
        'HomePlanet', 'Destination', 'CabinDeck', 'CabinSide',
        'VIP', 'CryoSleep', 'AgeGroup', 'Route', 'PassengerGroup'
    ]

    preserved_cat_features = {}

    for col in important_cat_cols:
        if col in combined.columns:
            if combined[col].dtype == 'object' or isinstance(combined[col].dtype, pd.CategoricalDtype):
                preserved_cat_features[col] = combined[col].copy()
                print(f"Preserving categorical feature: {col}")

    cols_to_drop = ['Name', 'Cabin', 'PassengerId']
    cols_to_drop = [col for col in cols_to_drop if col in combined.columns]
    combined = combined.drop(cols_to_drop, axis=1)

    cat_cols_to_encode = [
        col for col in combined.columns
        if col != 'source' and (
                combined[col].dtype == 'object' or
                isinstance(combined[col].dtype, pd.CategoricalDtype)
        )
    ]

    combined = pd.get_dummies(combined, columns=cat_cols_to_encode, drop_first=True)

    print(f"After preprocessing: {combined.shape[1] - 1} features created")

    for col, values in preserved_cat_features.items():
        combined[f'cat_{col}'] = values

    print(f"Added {len(preserved_cat_features)} preserved categorical features")

    X_train_proc = combined[combined['source'] == 'train'].drop('source', axis=1)
    X_val_proc = combined[combined['source'] == 'val'].drop('source', axis=1)
    X_test_proc = combined[combined['source'] == 'test'].drop('source', axis=1)

    all_cols = sorted(list(set(X_train_proc.columns) |
                           set(X_val_proc.columns) |
                           set(X_test_proc.columns)))

    for col in all_cols:
        if col not in X_train_proc.columns:
            X_train_proc[col] = 0
        if col not in X_val_proc.columns:
            X_val_proc[col] = 0
        if col not in X_test_proc.columns:
            X_test_proc[col] = 0

    for name, df in [('X_train_proc', X_train_proc), ('X_val_proc', X_val_proc), ('X_test_proc', X_test_proc)]:
        nan_cols = df.columns[df.isna().any()].tolist()

        if nan_cols:
            print(f"Found NaN values in {name} for columns: {nan_cols}")
            print("Filling remaining NaNs with appropriate values...")

            for col in nan_cols:
                if pd.api.types.is_numeric_dtype(df[col]):
                    fill_value = df[col].median()
                    if pd.isna(fill_value):
                        fill_value = 0
                    df[col] = df[col].fillna(fill_value)
                    print(f"  - {col}: filled with median ({fill_value})")
                else:
                    fill_value = df[col].mode().iloc[0] if not df[col].mode().empty else "Unknown"
                    df[col] = df[col].fillna(fill_value)
                    print(f"  - {col}: filled with mode ({fill_value})")

    if 'Age' in combined.columns:
        combined['Age_squared'] = combined['Age'] ** 2

    if 'Age' in combined.columns and 'CryoSleep' in combined.columns:
        combined['Age_CryoSleep'] = combined['Age'] * combined['CryoSleep'].astype(int)

    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    spending_cols = [col for col in spending_cols if col in combined.columns]

    if spending_cols and 'GroupSize' in combined.columns:
        combined['SpendPerPerson'] = combined['TotalSpend'] / combined['GroupSize']

        combined['SpendingDiversity'] = (combined[spending_cols] > 0).sum(axis=1)

        combined['SpendingPercentile'] = pd.qcut(combined['TotalSpend'].rank(method='first'),
                                                 q=10, labels=False, duplicates='drop')

    planet_map = {'Earth': 0, 'Europa': 1, 'Mars': 2}
    dest_map = {'TRAPPIST-1e': 0, 'PSO J318.5-22': 1, '55 Cancri e': 2}

    if 'HomePlanet' in combined.columns and 'Destination' in combined.columns:
        combined['HomePlanetNum'] = combined['HomePlanet'].map(planet_map)
        combined['DestinationNum'] = combined['Destination'].map(dest_map)

        combined['HomePlanetNum'] = combined['HomePlanetNum'].fillna(-1)
        combined['DestinationNum'] = combined['DestinationNum'].fillna(-1)

        combined['TravelDistance'] = abs(combined['DestinationNum'] - combined['HomePlanetNum'])

    if 'VIP' in combined.columns and 'TotalSpend' in combined.columns:
        combined['VIP_Spending'] = combined['VIP'].astype(int) * combined['TotalSpend']
    all_clean = all([
        not X_train_proc.isna().any().any(),
        not X_val_proc.isna().any().any(),
        not X_test_proc.isna().any().any()
    ])

    if all_clean:
        print("All datasets are now free of NaN values!")
    else:
        print("WARNING: There are still NaN values remaining. Check your data processing.")

    X_train_proc = X_train_proc[sorted(X_train_proc.columns)]
    X_val_proc = X_val_proc[sorted(X_val_proc.columns)]
    X_test_proc = X_test_proc[sorted(X_test_proc.columns)]

    preserved_categorical_cols = [f'cat_{col}' for col in preserved_cat_features.keys()]

    scaler = StandardScaler()

    cols_to_scale = [col for col in X_train_proc.columns if col not in preserved_categorical_cols]

    scaler.fit(X_train_proc[cols_to_scale])

    for name, df in [('X_train', X_train_proc), ('X_val', X_val_proc), ('X_test', X_test_proc)]:
        scaled_numerical = pd.DataFrame(
            scaler.transform(df[cols_to_scale]),
            columns=cols_to_scale,
            index=df.index
        )

        for col in cols_to_scale:
            df[col] = scaled_numerical[col]

    print("Data scaling completed with StandardScaler (preserved categorical features were NOT scaled)")
    print(f"Processed training features: {X_train_proc.shape}")
    print(f"Processed validation features: {X_val_proc.shape}")
    print(f"Processed test features: {X_test_proc.shape}")

    return {
        'X_train': X_train_proc,
        'X_val': X_val_proc,
        'X_test': X_test_proc,
        'y_train': y_train,
        'y_val': y_val,
        'train_ids': train_ids,
        'test_ids': test_ids,
        'preserved_categorical_cols': preserved_categorical_cols
    }

In [17]:
def train_base_models(data_dict, use_stratified_cv=True, cv_folds=5):
    from sklearn.model_selection import StratifiedKFold, cross_val_predict
    from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

    X_train = data_dict['X_train']
    X_val = data_dict['X_val']
    y_train = data_dict['y_train']
    y_val = data_dict['y_val']

    print("Training individual base models...")

    models = {
        'logistic': LogisticRegression(
            C=0.1,
            max_iter=1000,
            solver='liblinear',
            random_state=42
        ),
        'random_forest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
        'xgboost': xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42),
        'catboost': cb.CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, random_state=42, verbose=0)
    }

    results = {}

    if use_stratified_cv:
        skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        print(f"Using {cv_folds}-fold StratifiedKFold cross-validation")

    for name, model in models.items():
        print(f"\nTraining {name}...")

        model.fit(X_train, y_train)

        val_pred = model.predict(X_val)
        val_prob = model.predict_proba(X_val)[:, 1]

        accuracy = accuracy_score(y_val, val_pred)
        roc_auc = roc_auc_score(y_val, val_prob)

        if use_stratified_cv:
            cv_probs = cross_val_predict(
                model, X_train, y_train,
                cv=skf,
                method='predict_proba'
            )[:, 1]

            cv_auc = roc_auc_score(y_train, cv_probs)

            overfit_gap = roc_auc - cv_auc

            print(f"CV ROC AUC: {cv_auc:.4f}")
            print(f"Validation ROC AUC: {roc_auc:.4f}")
            print(f"Potential overfitting: {overfit_gap:.4f}")

            results[name] = {
                'model': model,
                'val_accuracy': accuracy,
                'val_auc': roc_auc,
                'cv_auc': cv_auc,
                'overfit_gap': overfit_gap,
                'val_predictions': val_pred,
                'val_probabilities': val_prob
            }
        else:
            results[name] = {
                'model': model,
                'val_accuracy': accuracy,
                'val_auc': roc_auc,
                'val_predictions': val_pred,
                'val_probabilities': val_prob
            }

        print(f"Validation Accuracy: {accuracy:.4f}")
        print(f"Validation ROC AUC: {roc_auc:.4f}")
        print("\nClassification Report:")
        print(classification_report(y_val, val_pred))

    comparison_columns = ['Model', 'Accuracy', 'ROC AUC']
    if use_stratified_cv:
        comparison_columns.extend(['CV AUC', 'Overfit Gap'])

    comparison_data = {'Model': list(results.keys())}
    comparison_data['Accuracy'] = [results[m]['val_accuracy'] for m in results]
    comparison_data['ROC AUC'] = [results[m]['val_auc'] for m in results]

    if use_stratified_cv:
        comparison_data['CV AUC'] = [results[m]['cv_auc'] for m in results]
        comparison_data['Overfit Gap'] = [results[m]['overfit_gap'] for m in results]

    comparison = pd.DataFrame(comparison_data).sort_values('ROC AUC', ascending=False)

    print("\n=== Model Comparison ===")
    print(comparison)

    return results

In [234]:
class StackingEnsemble(BaseEstimator, ClassifierMixin):

    def __init__(self, base_models, meta_learner):

        self.base_models = base_models
        self.meta_learner = meta_learner
        self.fitted_base_models = None
        self.fitted_meta_learner = None

    def fit(self, X, y):

        self.fitted_base_models = {}

        meta_features = np.zeros((X.shape[0], len(self.base_models)))

        for i, (name, model) in enumerate(self.base_models.items()):
            print(f"Training base model: {name}")

            model.fit(X, y)

            self.fitted_base_models[name] = model

            if hasattr(model, "predict_proba"):
                meta_features[:, i] = model.predict_proba(X)[:, 1]
            else:
                meta_features[:, i] = model.predict(X)

        print("Training meta-learner")
        self.fitted_meta_learner = self.meta_learner.fit(meta_features, y)

        return self

    def predict(self, X):

        meta_features = np.zeros((X.shape[0], len(self.base_models)))

        for i, (name, model) in enumerate(self.fitted_base_models.items()):
            if hasattr(model, "predict_proba"):
                meta_features[:, i] = model.predict_proba(X)[:, 1]
            else:
                meta_features[:, i] = model.predict(X)

        return self.fitted_meta_learner.predict(meta_features)

    def predict_proba(self, X):

        meta_features = np.zeros((X.shape[0], len(self.base_models)))

        for i, (name, model) in enumerate(self.fitted_base_models.items()):
            if hasattr(model, "predict_proba"):
                meta_features[:, i] = model.predict_proba(X)[:, 1]
            else:
                meta_features[:, i] = model.predict(X)

        return self.fitted_meta_learner.predict_proba(meta_features)

In [235]:
def build_advanced_ensemble(X_train, y_train, X_val, y_val, X_test):
    from sklearn.linear_model import LogisticRegression
    import xgboost as xgb
    import catboost as cb
    import numpy as np
    from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
    import optuna
    from sklearn.model_selection import StratifiedKFold, cross_val_score
    import os
    import time

    start_time = time.time()

    base_models = tune_base_models_with_optuna(
        X_train,
        y_train,
        cv=5,
        n_trials=50,
        timeout=1800,
        random_state=42
    )

    expected_models = ['logistic', 'xgboost', 'catboost']
    for model_name in expected_models:
        if model_name not in base_models:
            print(f"Warning: {model_name} is missing from base_models!")

    meta_features_train = np.zeros((X_train.shape[0], len(base_models)))
    meta_features_val = np.zeros((X_val.shape[0], len(base_models)))
    meta_features_test = np.zeros((X_test.shape[0], len(base_models)))

    used_models = []

    i = 0
    for name, model in base_models.items():
        print(f"Generating meta-features from {name}...")
        used_models.append(name)

        try:
            _ = model.predict(X_train[:1])
        except:
            print(f"Model {name} wasn't fitted, fitting now...")
            model.fit(X_train, y_train)

        if hasattr(model, "predict_proba"):
            meta_features_train[:, i] = model.predict_proba(X_train)[:, 1]
            meta_features_val[:, i] = model.predict_proba(X_val)[:, 1]
            meta_features_test[:, i] = model.predict_proba(X_test)[:, 1]
        else:
            meta_features_train[:, i] = model.predict(X_train)
            meta_features_val[:, i] = model.predict(X_val)
            meta_features_test[:, i] = model.predict(X_test)

        i += 1

    print(f"Generated meta-features from {len(used_models)} models: {', '.join(used_models)}")

    def optimize_meta_learner(model_type, X, y, cv=5):
        """Optimize meta-learner hyperparameters with Optuna"""
        cv_object = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

        if model_type == 'logistic':
            def objective(trial):
                C = trial.suggest_float('C', 0.001, 10.0, log=True)
                model = LogisticRegression(C=C, max_iter=2000, random_state=42)
                scores = cross_val_score(model, X, y, cv=cv_object, scoring='roc_auc')
                return scores.mean()

            study = optuna.create_study(direction='maximize')
            study.optimize(objective, n_trials=30)

            best_params = study.best_params
            print(f"Best logistic meta-learner params: {best_params}")
            return LogisticRegression(C=best_params['C'], max_iter=2000, random_state=42)

        elif model_type == 'xgb':
            def objective(trial):
                params = {
                    'n_estimators': trial.suggest_int('n_estimators', 50, 200),
                    'max_depth': trial.suggest_int('max_depth', 2, 5),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
                }
                model = xgb.XGBClassifier(**params, random_state=42)
                scores = cross_val_score(model, X, y, cv=cv_object, scoring='roc_auc')
                return scores.mean()

            study = optuna.create_study(direction='maximize')
            study.optimize(objective, n_trials=30)

            best_params = study.best_params
            print(f"Best XGBoost meta-learner params: {best_params}")
            return xgb.XGBClassifier(**best_params, random_state=42)

        elif model_type == 'catboost':
            def objective(trial):
                params = {
                    'iterations': trial.suggest_int('iterations', 50, 200),
                    'depth': trial.suggest_int('depth', 2, 5),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
                }
                model = cb.CatBoostClassifier(**params, random_state=42, verbose=0)
                scores = cross_val_score(model, X, y, cv=cv_object, scoring='roc_auc')
                return scores.mean()

            study = optuna.create_study(direction='maximize')
            study.optimize(objective, n_trials=30)

            best_params = study.best_params
            print(f"Best CatBoost meta-learner params: {best_params}")
            return cb.CatBoostClassifier(**best_params, random_state=42, verbose=0)

    meta_learners = {}
    for model_type in ['logistic', 'xgb', 'catboost']:
        print(f"Optimizing meta-learner: {model_type}...")
        optimized_model = optimize_meta_learner(model_type, meta_features_train, y_train)
        optimized_model.fit(meta_features_train, y_train)
        meta_learners[model_type] = optimized_model

    meta_preds_val = {}
    meta_preds_test = {}

    for name, model in meta_learners.items():
        print(f"Evaluating meta-learner: {name}...")

        if hasattr(model, "predict_proba"):
            val_pred = model.predict_proba(meta_features_val)[:, 1]
        else:
            val_pred = model.predict(meta_features_val)

        acc = accuracy_score(y_val, (val_pred > 0.5).astype(int))
        auc = roc_auc_score(y_val, val_pred)
        print(f"{name} - Accuracy: {acc:.4f}, AUC: {auc:.4f}")

        meta_preds_val[name] = val_pred
        meta_preds_test[name] = model.predict_proba(meta_features_test)[:, 1] if hasattr(model,
                                                                                         "predict_proba") else model.predict(
            meta_features_test)

    print("\nOptimizing ensemble weights...")

    def objective(trial):
        raw_weights = [
            trial.suggest_float(f'weight_{name}', 0.1, 1.0)
            for name in meta_preds_val.keys()
        ]

        weights_sum = sum(raw_weights)
        normalized_weights = [w / weights_sum for w in raw_weights]

        weighted_preds = np.zeros(len(X_val))
        for i, (name, pred) in enumerate(meta_preds_val.items()):
            weighted_preds += normalized_weights[i] * pred

        return roc_auc_score(y_val, weighted_preds)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    raw_weights = [
        study.best_params[f'weight_{name}']
        for name in meta_preds_val.keys()
    ]
    weights_sum = sum(raw_weights)
    weights = {
        name: raw_weights[i] / weights_sum
        for i, name in enumerate(meta_preds_val.keys())
    }

    print("\nOptimized meta-learner weights:")
    for name, weight in weights.items():
        print(f"{name}: {weight:.4f}")

    final_preds = np.zeros(len(X_test))
    for i, (name, pred) in enumerate(meta_preds_test.items()):
        final_preds += weights[name] * pred

    final_val_preds = np.zeros(len(X_val))
    for i, (name, pred) in enumerate(meta_preds_val.items()):
        final_val_preds += weights[name] * pred

    default_threshold = 0.5
    default_val_preds = (final_val_preds > default_threshold).astype(int)
    default_acc = accuracy_score(y_val, default_val_preds)
    ensemble_auc = roc_auc_score(y_val, final_val_preds)

    print("\n===== DEFAULT THRESHOLD EVALUATION =====")
    print(f"Default Threshold: {default_threshold}")
    print(f"Validation Accuracy: {default_acc:.4f}")
    print(f"Validation AUC: {ensemble_auc:.4f}")

    print("\nDefault Classification Report:")
    print(classification_report(y_val, default_val_preds))

    print("\n===== THRESHOLD OPTIMIZATION WITH OPTUNA =====")

    def threshold_objective(trial):
        threshold = trial.suggest_float('threshold', 0.3, 0.7)
        preds = (final_val_preds > threshold).astype(int)
        return accuracy_score(y_val, preds)

    threshold_study = optuna.create_study(direction='maximize')
    threshold_study.optimize(threshold_objective, n_trials=100)

    optimal_threshold = threshold_study.best_params['threshold']
    optimal_acc = threshold_study.best_value

    optimal_val_preds = (final_val_preds > optimal_threshold).astype(int)

    print(f"Optimal Threshold: {optimal_threshold:.4f}")
    print(f"Optimal Validation Accuracy: {optimal_acc:.4f}")
    print(f"Improvement: {(optimal_acc - default_acc) * 100:.4f}%")

    print("\nOptimized Classification Report:")
    print(classification_report(y_val, optimal_val_preds))

    optimal_test_preds = (final_preds > optimal_threshold).astype(int)

    elapsed = time.time() - start_time
    print(f"\nTotal ensemble building time: {elapsed:.2f} seconds ({elapsed / 60:.2f} minutes)")

    return {
        'base_models': base_models,
        'meta_learners': meta_learners,
        'weights': weights,
        'final_predictions': final_preds,
        'final_binary_predictions': {
            'default': (final_preds > default_threshold).astype(int),
            'optimized': optimal_test_preds
        },
        'validation_predictions': final_val_preds,
        'validation_binary_predictions': {
            'default': default_val_preds,
            'optimized': optimal_val_preds
        },
        'thresholds': {
            'default': default_threshold,
            'optimal': optimal_threshold
        },
        'meta_features': {
            'train': meta_features_train,
            'val': meta_features_val,
            'test': meta_features_test
        },
        'performance': {
            'default_accuracy': default_acc,
            'optimal_accuracy': optimal_acc,
            'improvement': optimal_acc - default_acc,
            'auc': ensemble_auc
        },
        'optuna_studies': {
            'base_models': None,
            'meta_weights': study,
            'threshold': threshold_study
        }
    }

In [236]:
def check_nan_values(data_dict):
    results = {}

    for key in ['X_train', 'X_val', 'X_test']:
        if key in data_dict and data_dict[key] is not None:
            df = data_dict[key]

            has_nan = df.isna().any().any()

            if has_nan:
                nan_cols = df.columns[df.isna().any()].tolist()
                nan_counts = {col: df[col].isna().sum() for col in nan_cols}
                nan_percent = {col: (count / len(df) * 100) for col, count in nan_counts.items()}

                results[key] = {
                    'has_nan': True,
                    'nan_columns': nan_cols,
                    'nan_counts': nan_counts,
                    'nan_percentages': nan_percent
                }

                print(f"\n{key} contains NaN values:")
                for col in nan_cols:
                    print(f"  - {col}: {nan_counts[col]} NaNs ({nan_percent[col]:.2f}%)")
            else:
                results[key] = {'has_nan': False}
                print(f"\n{key} does not contain any NaN values.")

    return results

In [237]:
def tune_base_models(X_train, y_train, cv=5, use_stratified_cv=True):
    from sklearn.model_selection import GridSearchCV

    print("Tuning model hyperparameters...")

    param_grids = {
        'logistic': {
            'C': [0.01, 0.1, 1.0, 10.0],
            'solver': ['liblinear'],
            'penalty': ['l1', 'l2']
        },
        # 'random_forest': {
        #     'n_estimators': [100, 200],
        #     'max_depth': [8, 10, 12, 15],
        #     'min_samples_split': [2, 5, 10],
        #     'min_samples_leaf': [1, 2, 4]
        # },
        'xgboost': {
            'n_estimators': [100, 200],
            'max_depth': [4, 5, 6],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0]
        },
        'catboost': {
            'iterations': [100, 200],
            'depth': [5, 6, 7],
            'learning_rate': [0.01, 0.1],
            'l2_leaf_reg': [1, 3, 5, 7]
        }
    }

    base_estimators = {
        'logistic': LogisticRegression(random_state=42, max_iter=2000),
        #'random_forest': RandomForestClassifier(random_state=42),
        'xgboost': xgb.XGBClassifier(random_state=42),
        'catboost': cb.CatBoostClassifier(random_state=42, verbose=0)
    }

    best_models = {}

    if use_stratified_cv:
        stratified_cv = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
        print(f"Using {cv}-fold StratifiedKFold cross-validation for hyperparameter tuning")
        cv_object = stratified_cv
    else:
        cv_object = cv

    for name, estimator in base_estimators.items():
        print(f"\nTuning {name}...")

        grid_search = GridSearchCV(
            estimator=estimator,
            param_grid=param_grids[name],
            cv=cv_object,
            scoring='roc_auc',
            n_jobs=os.cpu_count() - 1,
            verbose=2,
            return_train_score=True
        )

        grid_search.fit(X_train, y_train)
        elapsed = time.time() - start_time
        print(f"Grid search completed in {elapsed:.2f} seconds ({elapsed / 60:.2f} minutes)")
        print(f"Best parameters: {grid_search.best_params_}")
        print(f"Best score: {grid_search.best_score_:.4f}")

        best_index = grid_search.best_index_
        if hasattr(grid_search, 'cv_results_') and 'mean_train_score' in grid_search.cv_results_:
            train_score = grid_search.cv_results_['mean_train_score'][best_index]
            cv_score = grid_search.best_score_
            overfit_gap = train_score - cv_score
            print(f"Train score: {train_score:.4f}")
            print(f"CV score: {cv_score:.4f}")
            print(f"Potential overfitting: {overfit_gap:.4f}")

        best_models[name] = grid_search.best_estimator_

        print(f"Best {name} parameters: {grid_search.best_params_}")
        print(f"Best CV score: {grid_search.best_score_:.4f}")

    return best_models


def tune_base_models_with_optuna(
        X_train,
        y_train,
        cv=5,
        n_trials=200,
        timeout=None,
        random_state=42
):
    start_time = time.time()

    print("Tuning model hyperparameters with Optuna...")
    print(f"Using {cv}-fold StratifiedKFold cross-validation")

    best_models = {}

    cv_object = StratifiedKFold(n_splits=cv, shuffle=True, random_state=random_state)

    def optimize_logistic(trial):
        C = trial.suggest_float('C', 0.001, 100, log=True)
        penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
        solver = 'liblinear'

        model = LogisticRegression(
            C=C,
            penalty=penalty,
            solver=solver,
            random_state=random_state,
            max_iter=2000
        )

        scores = cross_val_score(
            model, X_train, y_train,
            cv=cv_object,
            scoring='roc_auc',
            n_jobs=os.cpu_count() - 1
        )

        return scores.mean()

    def optimize_xgboost(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0.0, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True)
        }

        model = xgb.XGBClassifier(
            **params,
            random_state=random_state,
            use_label_encoder=False,
            eval_metric='logloss'
        )

        scores = cross_val_score(
            model, X_train, y_train,
            cv=cv_object,
            scoring='roc_auc',
            n_jobs=os.cpu_count() - 1
        )

        return scores.mean()

    def optimize_catboost(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 50, 300),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 10.0, log=True),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 10.0),
            'random_strength': trial.suggest_float('random_strength', 1e-9, 10.0, log=True),
            'border_count': trial.suggest_int('border_count', 32, 255),
            'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
        }

        model = cb.CatBoostClassifier(
            **params,
            random_state=random_state,
            verbose=0
        )

        scores = cross_val_score(
            model, X_train, y_train,
            cv=cv_object,
            scoring='roc_auc',
            n_jobs=os.cpu_count() - 1
        )

        return scores.mean()

    optimization_funcs = {
        'logistic': optimize_logistic,
        'xgboost': optimize_xgboost,
        'catboost': optimize_catboost
    }

    for model_name, objective_func in optimization_funcs.items():
        print(f"\nTuning {model_name}...")
        model_start_time = time.time()

        study = optuna.create_study(direction='maximize')
        study.optimize(objective_func, n_trials=n_trials, timeout=timeout)

        model_elapsed = time.time() - model_start_time
        print(f"Optimization completed in {model_elapsed:.2f} seconds ({model_elapsed / 60:.2f} minutes)")

        best_params = study.best_params
        best_score = study.best_value
        print(f"Best parameters: {best_params}")
        print(f"Best CV score: {best_score:.4f}")

        if model_name == 'logistic':
            best_model = LogisticRegression(
                C=best_params['C'],
                penalty=best_params['penalty'],
                solver='liblinear',
                random_state=random_state,
                max_iter=2000
            )
        elif model_name == 'xgboost':
            best_model = xgb.XGBClassifier(
                **best_params,
                random_state=random_state,
                use_label_encoder=False,
                eval_metric='logloss'
            )
        elif model_name == 'catboost':
            best_model = cb.CatBoostClassifier(
                **best_params,
                random_state=random_state,
                verbose=0
            )

        best_model.fit(X_train, y_train)

        if hasattr(best_model, "predict_proba"):
            train_preds = best_model.predict_proba(X_train)[:, 1]
        else:
            train_preds = best_model.predict(X_train)

        from sklearn.metrics import roc_auc_score
        train_score = roc_auc_score(y_train, train_preds)
        cv_score = best_score
        overfit_gap = train_score - cv_score

        print(f"Train score: {train_score:.4f}")
        print(f"CV score: {cv_score:.4f}")
        print(f"Potential overfitting: {overfit_gap:.4f}")

        best_models[model_name] = best_model

        print(f"Number of finished trials: {len(study.trials)}")
        print(f"Best {model_name} parameters: {best_params}")
        print(f"Best CV score: {best_score:.4f}")

        optuna.visualization.plot_optimization_history(study)
        plt.title(f"{model_name} Optimization History")
        plt.show()

    total_elapsed = time.time() - start_time
    print(f"\nTotal hyperparameter tuning time: {total_elapsed:.2f} seconds ({total_elapsed / 60:.2f} minutes)")

    return best_models

In [238]:
def select_best_features(X_train, y_train, X_val, X_test):
    from sklearn.feature_selection import SelectFromModel
    import xgboost as xgb

    print("Performing feature selection...")

    model = xgb.XGBClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    selection = SelectFromModel(model, threshold='median', prefit=True).set_output(transform="pandas")

    X_train_selected = selection.transform(X_train)
    X_val_selected = selection.transform(X_val)
    X_test_selected = selection.transform(X_test)

    print(f"Selected {X_train_selected.shape[1]} features out of {X_train.shape[1]}")

    feature_importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': model.feature_importances_
    })
    top_features = feature_importance.sort_values('Importance', ascending=False).head(10)
    print("Top 10 most important features:")
    print(top_features)

    return X_train_selected, X_val_selected, X_test_selected

In [239]:
def optimize_threshold(y_true, y_prob, metric='accuracy', thresholds=None):
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
    import numpy as np

    if thresholds is None:
        thresholds = np.arange(0.3, 0.7, 0.01)

    best_threshold = 0.5
    best_score = 0.0

    for threshold in thresholds:
        y_pred = (y_prob > threshold).astype(int)

        if metric == 'accuracy':
            score = accuracy_score(y_true, y_pred)
        elif metric == 'f1':
            score = f1_score(y_true, y_pred)
        elif metric == 'precision':
            score = precision_score(y_true, y_pred)
        elif metric == 'recall':
            score = recall_score(y_true, y_pred)
        else:
            score = accuracy_score(y_true, y_pred)

        if score > best_score:
            best_score = score
            best_threshold = threshold

    return best_threshold, best_score


In [22]:
def enhanced_feature_engineering(combined):
    """Apply advanced feature engineering techniques"""
    print("Adding enhanced features...")

    if 'Age' in combined.columns and 'TotalSpend' in combined.columns:
        combined['Age_TotalSpend'] = combined['Age'] * combined['TotalSpend']
        combined['Age_TotalSpendRatio'] = combined['Age'] / (combined['TotalSpend'] + 1)

    if 'Age' in combined.columns:
        age_valid = combined['Age'] >= 0
        combined['Age_log'] = np.nan
        combined['Age_sqrt'] = np.nan

        combined.loc[age_valid, 'Age_log'] = np.log1p(combined.loc[age_valid, 'Age'])
        combined.loc[age_valid, 'Age_sqrt'] = np.sqrt(combined.loc[age_valid, 'Age'])

    if 'PassengerGroup' in combined.columns and 'TotalSpend' in combined.columns:
        group_spend_mean = combined.groupby('PassengerGroup')['TotalSpend'].transform('mean')
        group_spend_std = combined.groupby('PassengerGroup')['TotalSpend'].transform('std').fillna(0)
        combined['GroupSpendDeviation'] = (combined['TotalSpend'] - group_spend_mean) / (group_spend_std + 1)

    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    spending_cols = [col for col in spending_cols if col in combined.columns]
    if spending_cols:
        for col in spending_cols:
            other_cols = [c for c in spending_cols if c != col]
            if other_cols:
                combined[f'{col}_Dominance'] = combined[col] / (combined[other_cols].sum(axis=1) + 1)

    return combined


def advanced_feature_selection(X_train, y_train, X_val, X_test, selection_method='combined', n_features=None):
    """
    Advanced feature selection using multiple methods with NaN handling
    """
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_selection import mutual_info_classif
    from sklearn.impute import SimpleImputer

    print(f"Performing feature selection using {selection_method} method...")

    if X_train.isna().any().any():
        print("Detected NaN values in training data. Applying imputation...")

        imputer = SimpleImputer(strategy='median')
        X_train_imputed = pd.DataFrame(
            imputer.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )

        X_val_imputed = pd.DataFrame(
            imputer.transform(X_val),
            columns=X_val.columns,
            index=X_val.index
        )

        X_test_imputed = pd.DataFrame(
            imputer.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )
    else:
        X_train_imputed = X_train
        X_val_imputed = X_val
        X_test_imputed = X_test

    if selection_method == 'importance' or selection_method == 'combined':
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train_imputed, y_train)

        importances = pd.Series(model.feature_importances_, index=X_train_imputed.columns)

    if selection_method == 'mutual_info' or selection_method == 'combined':
        mi_scores = pd.Series(
            mutual_info_classif(X_train_imputed, y_train, random_state=42),
            index=X_train_imputed.columns
        )

    if selection_method == 'combined':
        importances = (importances - importances.min()) / (importances.max() - importances.min())
        mi_scores = (mi_scores - mi_scores.min()) / (mi_scores.max() - mi_scores.min())

        combined_scores = (importances + mi_scores) / 2
        scores = combined_scores
    elif selection_method == 'importance':
        scores = importances
    else:
        scores = mi_scores

    sorted_features = scores.sort_values(ascending=False)

    if n_features is not None:
        selected = sorted_features.index[:n_features].tolist()
    else:
        threshold = sorted_features.mean()
        selected = sorted_features[sorted_features > threshold].index.tolist()

    print(f"Selected {len(selected)} features")
    if len(selected) > 0:
        print(f"Top 5 features: {selected[:5]}")

    return X_train_imputed[selected], X_val_imputed[selected], X_test_imputed[selected], selected

In [241]:
def build_stacked_ensemble(X_train, y_train, X_val, y_val, X_test, n_folds=5):
    """
    Build a stacked ensemble model using out-of-fold predictions
    """
    from sklearn.model_selection import StratifiedKFold
    from sklearn.base import clone
    from sklearn.metrics import roc_auc_score, accuracy_score
    from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
    from sklearn.linear_model import LogisticRegression
    import xgboost as xgb
    import catboost as cb
    import numpy as np

    def get_oof_predictions(model, X, y, X_test, n_folds):
        skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
        oof = np.zeros(len(X))
        test_preds = np.zeros(len(X_test))

        for i, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            print(f"Training fold {i + 1}/{n_folds}")
            X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
            y_train_fold = y.iloc[train_idx]

            model_clone = clone(model)
            model_clone.fit(X_train_fold, y_train_fold)

            if hasattr(model, 'predict_proba'):
                oof[val_idx] = model_clone.predict_proba(X_val_fold)[:, 1]
                test_preds += model_clone.predict_proba(X_test)[:, 1] / n_folds
            else:
                oof[val_idx] = model_clone.predict(X_val_fold)
                test_preds += model_clone.predict(X_test) / n_folds

        return oof, test_preds

    base_models = {
        'catboost': cb.CatBoostClassifier(iterations=500, depth=6, learning_rate=0.05, verbose=False, random_seed=42),
        'xgboost': xgb.XGBClassifier(n_estimators=500, max_depth=4, learning_rate=0.05, random_state=42),
        'extratrees': ExtraTreesClassifier(n_estimators=500, max_depth=12, random_state=42),
        'gbm': GradientBoostingClassifier(n_estimators=300, max_depth=3, learning_rate=0.05, random_state=42),
        'logistic': LogisticRegression(C=0.1, max_iter=1000, random_state=42)
    }

    meta_features_train = np.zeros((X_train.shape[0], len(base_models)))
    meta_features_val = np.zeros((X_val.shape[0], len(base_models)))
    meta_features_test = np.zeros((X_test.shape[0], len(base_models)))

    for i, (name, model) in enumerate(base_models.items()):
        print(f"Building meta-features for {name}")
        oof_preds, test_preds = get_oof_predictions(model, X_train, y_train, X_test, n_folds)
        meta_features_train[:, i] = oof_preds

        model.fit(X_train, y_train)
        meta_features_val[:, i] = model.predict_proba(X_val)[:, 1] if hasattr(model,
                                                                              'predict_proba') else model.predict(X_val)
        meta_features_test[:, i] = test_preds

        oof_auc = roc_auc_score(y_train, oof_preds)
        print(f"{name} OOF AUC: {oof_auc:.4f}")

    meta_model = cb.CatBoostClassifier(iterations=300, depth=4, learning_rate=0.03, verbose=False, random_seed=42)
    meta_model.fit(meta_features_train, y_train)

    val_probs = meta_model.predict_proba(meta_features_val)[:, 1]
    test_probs = meta_model.predict_proba(meta_features_test)[:, 1]

    train_auc = roc_auc_score(y_train, meta_model.predict_proba(meta_features_train)[:, 1])
    val_auc = roc_auc_score(y_val, val_probs)

    default_threshold = 0.5
    default_val_preds = (val_probs > default_threshold).astype(int)
    default_accuracy = accuracy_score(y_val, default_val_preds)

    thresholds = np.linspace(0.3, 0.7, 40)
    best_threshold = 0.5
    best_accuracy = default_accuracy

    for threshold in thresholds:
        val_preds = (val_probs > threshold).astype(int)
        acc = accuracy_score(y_val, val_preds)
        if acc > best_accuracy:
            best_accuracy = acc
            best_threshold = threshold

    optimized_val_preds = (val_probs > best_threshold).astype(int)

    print(f"Meta-model training AUC: {train_auc:.4f}")
    print(f"Meta-model validation AUC: {val_auc:.4f}")
    print(f"Default accuracy: {default_accuracy:.4f}")
    print(f"Optimal threshold: {best_threshold:.4f}")
    print(f"Optimized accuracy: {best_accuracy:.4f}")

    return {
        'meta_model': meta_model,
        'base_models': base_models,
        'final_predictions': test_probs,
        'validation_predictions': {
            'default': val_probs,
            'optimized': optimized_val_preds
        },
        'validation_binary_predictions': {
            'default': default_val_preds,
            'optimized': optimized_val_preds
        },
        'thresholds': {
            'default': default_threshold,
            'optimal': best_threshold
        },
        'performance': {
            'default_accuracy': default_accuracy,
            'optimal_accuracy': best_accuracy,
            'auc': val_auc
        }
    }

In [242]:
def select_top_features(X_train, y_train, n_features=None, method='importance'):
    """
    Select top features based on importance from a tree-based model
    
    Parameters:
    -----------
    X_train : DataFrame
        Training features
    y_train : Series
        Target values
    n_features : int, optional
        Number of features to select. If None, selects based on importance threshold
    method : str
        Method for feature selection ('importance', 'mutual_info', or 'combined')
        
    Returns:
    --------
    list
        Names of selected features
    """
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_selection import mutual_info_classif

    if method == 'importance' or method == 'combined':
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        importances = pd.Series(model.feature_importances_, index=X_train.columns)

    if method == 'mutual_info' or method == 'combined':
        mi_scores = pd.Series(
            mutual_info_classif(X_train, y_train, random_state=42),
            index=X_train.columns
        )

    if method == 'combined':

        importances = (importances - importances.min()) / (importances.max() - importances.min())
        mi_scores = (mi_scores - mi_scores.min()) / (mi_scores.max() - mi_scores.min())

        combined_scores = (importances + mi_scores) / 2
        scores = combined_scores
    elif method == 'importance':
        scores = importances
    else:
        scores = mi_scores

    sorted_features = scores.sort_values(ascending=False)

    if n_features is not None:
        selected = sorted_features.index[:n_features].tolist()
    else:
        threshold = sorted_features.mean()
        selected = sorted_features[sorted_features > threshold].index.tolist()

    return selected


def optimize_threshold(predictions, y_true, metric='accuracy'):
    """
    Find the optimal threshold to maximize a given metric
    
    Parameters:
    -----------
    predictions : array-like
        Predicted probabilities
    y_true : array-like
        True labels
    metric : str
        Metric to optimize ('accuracy', 'f1', 'precision', 'recall')
        
    Returns:
    --------
    float
        Optimal threshold
    """
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

    metric_funcs = {
        'accuracy': accuracy_score,
        'f1': f1_score,
        'precision': precision_score,
        'recall': recall_score
    }

    metric_func = metric_funcs.get(metric, accuracy_score)

    thresholds = np.arange(0.01, 1.0, 0.01)

    best_score = 0
    best_threshold = 0.5

    for threshold in thresholds:
        pred_labels = (predictions > threshold).astype(int)
        score = metric_func(y_true, pred_labels)

        if score > best_score:
            best_score = score
            best_threshold = threshold

    print(f"Optimized threshold: {best_threshold:.4f} with {metric} score: {best_score:.4f}")
    return best_threshold

In [243]:
def enhanced_spaceship_data_prep():
    data_dict = prepare_spaceship_data(train_data, test_data)

    X_train = data_dict['X_train']
    X_val = data_dict['X_val']
    X_test = data_dict['X_test']

    best_features = select_top_features(X_train, data_dict['y_train'])

    data_dict['X_train'] = X_train[best_features]
    data_dict['X_val'] = X_val[best_features]
    data_dict['X_test'] = X_test[best_features]

    return data_dict

In [24]:
import optuna
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import catboost as cb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import time


def tune_with_optuna_multi_data(data_dict, categorical_cols=None, n_trials=100, n_jobs=1):
    """
    Comprehensive Optuna tuning for ensemble model with model-specific data formats
    
    Parameters:
    -----------
    data_dict : dict
        Dictionary containing all training and validation data
    categorical_cols : list, default=None
        List of categorical column names for CatBoost
    n_trials : int, default=100
        Number of trials for hyperparameter optimization
    n_jobs : int, default=1
        Number of parallel jobs. -1 for using all processors.
    """
    print("Starting comprehensive Optuna tuning with model-specific data formats...")

    X_train = data_dict['X_train']
    y_train = data_dict['y_train']
    X_val = data_dict['X_val']
    y_val = data_dict['y_val']

    print("Creating model-specific datasets...")

    X_train_catboost = X_train.copy()
    X_val_catboost = X_val.copy()

    if categorical_cols:
        for col in categorical_cols:
            if col in X_train_catboost.columns:
                X_train_catboost[col] = X_train_catboost[col].astype(str)
                X_val_catboost[col] = X_val_catboost[col].astype(str)

    X_train_xgb = X_train.copy()
    X_val_xgb = X_val.copy()

    if categorical_cols:
        for col in categorical_cols:
            if col in X_train_xgb.columns:
                X_train_xgb[col] = X_train_xgb[col].astype('category')
                X_val_xgb[col] = X_val_xgb[col].astype('category')

    X_train_sklearn = X_train.copy()
    X_val_sklearn = X_val.copy()

    cat_cols_to_encode = []
    for col in X_train_sklearn.columns:
        if X_train_sklearn[col].dtype == 'object' or isinstance(X_train_sklearn[col].dtype, pd.CategoricalDtype):
            cat_cols_to_encode.append(col)

    if cat_cols_to_encode:
        from sklearn.preprocessing import OrdinalEncoder
        encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

        cat_encoded = encoder.fit_transform(X_train_sklearn[cat_cols_to_encode])
        for i, col in enumerate(cat_cols_to_encode):
            X_train_sklearn[col] = cat_encoded[:, i]

        cat_encoded_val = encoder.transform(X_val_sklearn[cat_cols_to_encode])
        for i, col in enumerate(cat_cols_to_encode):
            X_val_sklearn[col] = cat_encoded_val[:, i]

    print(f"Created specialized datasets for each model type")

    base_models = {}

    if categorical_cols and len(categorical_cols) > 0:
        print(f"First 5 categorical features: {categorical_cols[:5]}")

    catboost_model, catboost_score = tune_catboost(
        X_train_catboost, y_train, X_val_catboost, y_val,
        categorical_cols=categorical_cols,
        n_trials=n_trials,
        n_jobs=n_jobs
    )
    base_models['catboost'] = catboost_model
    print(f"Best CatBoost validation accuracy: {catboost_score:.4f}")

    print(f"\n==== Tuning XGBoost ({n_jobs} parallel jobs) ====")
    print(f"Tuning XGBoost with {n_jobs} threads")
    xgb_model, xgb_score = tune_xgboost(
        X_train_xgb, y_train, X_val_xgb, y_val,
        n_trials=n_trials,
        n_jobs=n_jobs
    )
    base_models['xgboost'] = xgb_model
    print(f"Best XGBoost validation accuracy: {xgb_score:.4f}")

    print(f"\n==== Tuning RandomForest ({n_jobs} parallel jobs) ====")
    rf_model, rf_score = tune_random_forest(
        X_train_sklearn, y_train, X_val_sklearn, y_val,
        n_trials=n_trials,
        n_jobs=n_jobs
    )
    base_models['random_forest'] = rf_model
    print(f"Best RandomForest validation accuracy: {rf_score:.4f}")

    print(f"\n==== Tuning GradientBoosting ({n_jobs} parallel jobs) ====")
    gb_model, gb_score = tune_gradient_boosting(
        X_train_sklearn, y_train, X_val_sklearn, y_val,
        n_trials=n_trials,
        n_jobs=n_jobs
    )
    base_models['gradient_boosting'] = gb_model
    print(f"Best GradientBoosting validation accuracy: {gb_score:.4f}")

    print(f"\n==== Tuning ExtraTrees ({n_jobs} parallel jobs) ====")
    et_model, et_score = tune_extra_trees(
        X_train_sklearn, y_train, X_val_sklearn, y_val,
        n_trials=n_trials,
        n_jobs=n_jobs
    )
    base_models['extra_trees'] = et_model
    print(f"Best ExtraTrees validation accuracy: {et_score:.4f}")

    data_dict['X_test_catboost'] = data_dict['X_test'].copy()
    data_dict['X_test_xgb'] = data_dict['X_test'].copy()
    data_dict['X_test_sklearn'] = data_dict['X_test'].copy()

    if categorical_cols:
        for col in categorical_cols:
            if col in data_dict['X_test_catboost'].columns:
                data_dict['X_test_catboost'][col] = data_dict['X_test_catboost'][col].astype(str)
                data_dict['X_test_xgb'][col] = data_dict['X_test_xgb'][col].astype('category')

    if cat_cols_to_encode:
        cat_encoded_test = encoder.transform(data_dict['X_test_sklearn'][cat_cols_to_encode])
        for i, col in enumerate(cat_cols_to_encode):
            data_dict['X_test_sklearn'][col] = cat_encoded_test[:, i]

    print("\n==== Generating Meta Features with Model-Specific Data ====")
    meta_features_train = np.zeros((X_train.shape[0], len(base_models)))
    meta_preds_val = np.zeros((X_val.shape[0], len(base_models)))

    model_data_map = {
        'catboost': (X_train_catboost, X_val_catboost),
        'xgboost': (X_train_xgb, X_val_xgb),
        'random_forest': (X_train_sklearn, X_val_sklearn),
        'gradient_boosting': (X_train_sklearn, X_val_sklearn),
        'extra_trees': (X_train_sklearn, X_val_sklearn)
    }

    for i, (name, model) in enumerate(base_models.items()):
        print(f"Generating meta-features for {name}")

        X_train_model, X_val_model = model_data_map[name]

        from sklearn.model_selection import StratifiedKFold
        from sklearn.base import clone

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        oof_preds = np.zeros(X_train.shape[0])

        for train_idx, val_idx in cv.split(X_train_model, y_train):
            X_fold_train = X_train_model.iloc[train_idx]
            X_fold_val = X_train_model.iloc[val_idx]
            y_fold_train = y_train.iloc[train_idx]

            model_clone = clone(model)

            if name == 'catboost':
                model_clone.fit(
                    X_fold_train,
                    y_fold_train,
                    cat_features=categorical_cols,
                    verbose=False
                )
            else:
                model_clone.fit(X_fold_train, y_fold_train)

            if hasattr(model_clone, 'predict_proba'):
                fold_preds = model_clone.predict_proba(X_fold_val)[:, 1]
            else:
                fold_preds = model_clone.predict(X_fold_val)

            oof_preds[val_idx] = fold_preds

        meta_features_train[:, i] = oof_preds

        if hasattr(model, 'predict_proba'):
            val_preds = model.predict_proba(X_val_model)[:, 1]
        else:
            val_preds = model.predict(X_val_model)

        meta_preds_val[:, i] = val_preds

    print(f"\n==== Tuning Meta-Learner ({n_jobs} parallel jobs) ====")
    meta_learner, meta_score = tune_meta_learner(
        meta_features_train, y_train, meta_preds_val, y_val,
        n_trials=n_trials,
        n_jobs=n_jobs
    )
    print(f"Best meta-learner validation accuracy: {meta_score:.4f}")

    ensemble = {
        'base_models': base_models,
        'meta_learner': meta_learner,
        'model_data_map': model_data_map,
        'meta_score': meta_score
    }

    return ensemble


def tune_catboost(X_train, y_train, X_val, y_val, categorical_cols=None, n_trials=50, n_jobs=1):
    """
    Tune CatBoost hyperparameters with proper categorical feature handling and parallel processing
    """
    if categorical_cols is None:
        categorical_cols = []

    print(f"Tuning CatBoost with {len(categorical_cols)} categorical features")
    print(f"First 5 categorical features: {categorical_cols[:5]}")

    def objective(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 800, 2000),
            'depth': trial.suggest_int('depth', 6, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 10.0, log=True),
            'random_strength': trial.suggest_float('random_strength', 0.1, 10.0, log=True),
            'bootstrap_type': 'Bayesian',
            'bagging_temperature': trial.suggest_float('bagging_temperature', 1.0, 10.0),
            'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise']),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 20),
            'max_ctr_complexity': trial.suggest_int('max_ctr_complexity', 1, 4),
            'one_hot_max_size': trial.suggest_int('one_hot_max_size', 10, 255),
            'verbose': 0,
            'thread_count': n_jobs if n_jobs > 0 else None,
            'random_seed': 42
        }

        model = cb.CatBoostClassifier(**params)

        model.fit(
            X_train, y_train,
            cat_features=categorical_cols,
            eval_set=(X_val, y_val),
            early_stopping_rounds=50,
            verbose=False
        )

        preds = model.predict_proba(X_val)[:, 1]
        thresholds = np.linspace(0.4, 0.6, 20)
        best_acc = 0

        for threshold in thresholds:
            binary_preds = (preds > threshold).astype(int)
            acc = accuracy_score(y_val, binary_preds)
            if acc > best_acc:
                best_acc = acc

        return best_acc

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs)

    print(f"Best CatBoost parameters: {study.best_params}")

    best_params = study.best_params.copy()
    best_params['random_seed'] = 42
    best_params['verbose'] = 100
    best_params['thread_count'] = n_jobs if n_jobs > 0 else None

    best_model = cb.CatBoostClassifier(**best_params)
    best_model.fit(
        X_train, y_train,
        cat_features=categorical_cols,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50
    )

    best_model.categorical_features = categorical_cols

    val_preds = best_model.predict_proba(X_val)[:, 1]
    thresholds = np.linspace(0.3, 0.7, 40)
    best_acc = 0
    best_threshold = 0.5

    for threshold in thresholds:
        binary_preds = (val_preds > threshold).astype(int)
        acc = accuracy_score(y_val, binary_preds)
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold

    print(f"Best accuracy: {best_acc:.4f} with threshold: {best_threshold:.4f}")
    best_model.threshold = best_threshold

    return best_model, best_acc


def tune_xgboost(X_train, y_train, X_val, y_val, n_trials=50, n_jobs=1):
    """
    Tune XGBoost hyperparameters with proper categorical feature handling
    """
    import xgboost as xgb
    import pandas as pd

    print(f"Tuning XGBoost with {n_jobs} threads")

    if not isinstance(X_train, pd.DataFrame):
        X_train = pd.DataFrame(X_train)
    if not isinstance(X_val, pd.DataFrame):
        X_val = pd.DataFrame(X_val)

    for col in X_train.select_dtypes(include=['object']).columns:
        X_train[col] = X_train[col].astype('category')
        X_val[col] = X_val[col].astype('category')

    print(f"Converted {len(X_train.select_dtypes(include=['category']).columns)} columns to category type")

    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0.01, 1.0, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10.0, log=True),
            'n_jobs': n_jobs,
            'enable_categorical': True
        }

        model = xgb.XGBClassifier(**params, random_state=42)

        try:
            callbacks = [xgb.callback.EarlyStopping(rounds=50, verbose=False)]
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks, verbose=False)
        except:
            try:
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                          early_stopping_rounds=50, verbose=False)
            except:

                model.fit(X_train, y_train)

        preds = model.predict_proba(X_val)[:, 1]
        thresholds = np.linspace(0.4, 0.6, 20)
        best_acc = 0

        for threshold in thresholds:
            binary_preds = (preds > threshold).astype(int)
            acc = accuracy_score(y_val, binary_preds)
            if acc > best_acc:
                best_acc = acc

        return best_acc

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials,
                   n_jobs=1)
    print(f"Best XGBoost parameters: {study.best_params}")

    best_params = study.best_params.copy()
    best_params['random_state'] = 42
    best_params['n_jobs'] = n_jobs
    best_params['enable_categorical'] = True

    for col in X_train.select_dtypes(include=['object']).columns:
        X_train[col] = X_train[col].astype('category')
        X_val[col] = X_val[col].astype('category')

    best_model = xgb.XGBClassifier(**best_params)

    try:
        callbacks = [xgb.callback.EarlyStopping(rounds=50, verbose=True)]
        best_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
    except:
        try:
            best_model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                           early_stopping_rounds=50, verbose=True)
        except:
            best_model.fit(X_train, y_train)

    val_preds = best_model.predict_proba(X_val)[:, 1]
    thresholds = np.linspace(0.3, 0.7, 40)
    best_acc = 0
    best_threshold = 0.5

    for threshold in thresholds:
        binary_preds = (val_preds > threshold).astype(int)
        acc = accuracy_score(y_val, binary_preds)
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold

    print(f"Best accuracy: {best_acc:.4f} with threshold: {best_threshold:.4f}")
    best_model.threshold = best_threshold

    return best_model, best_acc


def tune_random_forest(X_train, y_train, X_val, y_val, n_trials=50, n_jobs=1):
    """
    Tune RandomForest hyperparameters with parallel processing
    
    Parameters:
    -----------
    X_train, y_train, X_val, y_val : training and validation data
    n_trials : int, number of Optuna trials
    n_jobs : int, number of parallel jobs (-1 for all cores)
    """

    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 5, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
            'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
            'n_jobs': n_jobs,
            'random_state': 42
        }

        model = RandomForestClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return accuracy_score(y_val, y_pred)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, timeout=600, n_jobs=n_jobs)

    print(f"Best RandomForest parameters: {study.best_params}")

    best_params = study.best_params.copy()
    best_params['n_jobs'] = n_jobs
    best_model = RandomForestClassifier(**best_params)
    best_model.fit(X_train, y_train)

    best_score = accuracy_score(y_val, best_model.predict(X_val))
    print(f"Best accuracy: {best_score:.4f}")
    return best_model, best_score


def tune_gradient_boosting(X_train, y_train, X_val, y_val, n_trials=50, n_jobs=1):
    """
    Tune GradientBoosting hyperparameters with early stopping
    
    Note: GradientBoostingClassifier doesn't support n_jobs internally,
    but we can use parallelization for the trials.
    """

    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'random_state': 42
        }

        model = GradientBoostingClassifier(**params)
        model.fit(X_train, y_train)

        staged_preds = list(model.staged_predict(X_val))
        best_iter = np.argmax([accuracy_score(y_val, pred) for pred in staged_preds])

        best_score = accuracy_score(y_val, staged_preds[best_iter])

        return best_score

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, timeout=600, n_jobs=n_jobs)

    print(f"Best GradientBoosting parameters: {study.best_params}")

    best_params = study.best_params.copy()
    best_model = GradientBoostingClassifier(**best_params)
    best_model.fit(X_train, y_train)

    staged_preds = list(best_model.staged_predict(X_val))
    best_iter = np.argmax([accuracy_score(y_val, pred) for pred in staged_preds])
    print(f"Best iteration for GradientBoosting: {best_iter + 1}/{best_params['n_estimators']}")

    if best_iter + 1 < best_params['n_estimators']:
        print(f"Pruning model to {best_iter + 1} trees (early stopping)")
        final_model = GradientBoostingClassifier(
            n_estimators=best_iter + 1,
            learning_rate=best_params['learning_rate'],
            max_depth=best_params['max_depth'],
            min_samples_split=best_params['min_samples_split'],
            min_samples_leaf=best_params['min_samples_leaf'],
            subsample=best_params['subsample'],
            max_features=best_params['max_features'],
            random_state=42
        )
        final_model.fit(X_train, y_train)
        best_model = final_model

    best_score = accuracy_score(y_val, best_model.predict(X_val))
    print(f"Best accuracy: {best_score:.4f}")
    return best_model, best_score


def tune_extra_trees(X_train, y_train, X_val, y_val, n_trials=50, n_jobs=1):
    """
    Tune ExtraTrees hyperparameters with parallel processing
    """

    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 5, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
            'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
            'n_jobs': n_jobs,
            'random_state': 42
        }

        model = ExtraTreesClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        return accuracy_score(y_val, y_pred)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, timeout=600, n_jobs=n_jobs)

    print(f"Best ExtraTrees parameters: {study.best_params}")

    best_params = study.best_params.copy()
    best_params['n_jobs'] = n_jobs
    best_model = ExtraTreesClassifier(**best_params)
    best_model.fit(X_train, y_train)

    best_score = accuracy_score(y_val, best_model.predict(X_val))
    print(f"Best accuracy: {best_score:.4f}")
    return best_model, best_score


def generate_meta_features(base_models, X_train, y_train, X_val, categorical_cols=None):
    """Generate meta-features for stacking with support for categorical features"""
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    meta_features_train = np.zeros((X_train.shape[0], len(base_models)))

    for i, (name, model) in enumerate(base_models.items()):
        print(f"Generating OOF predictions for {name}")
        oof_preds = np.zeros(X_train.shape[0])

        for train_idx, val_idx in cv.split(X_train, y_train):
            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train = y_train.iloc[train_idx]

            model_clone = model.__class__(**model.get_params())

            if name == 'catboost' and hasattr(model, 'categorical_features'):
                model_clone.fit(
                    X_fold_train, y_fold_train,
                    cat_features=model.categorical_features,
                    verbose=False
                )

                fold_preds = model_clone.predict_proba(X_fold_val)[:, 1]
            else:
                model_clone.fit(X_fold_train, y_fold_train)

                if hasattr(model_clone, 'predict_proba'):
                    fold_preds = model_clone.predict_proba(X_fold_val)[:, 1]
                else:
                    fold_preds = model_clone.predict(X_fold_val)

            oof_preds[val_idx] = fold_preds

        meta_features_train[:, i] = oof_preds

    meta_preds_val = np.zeros((X_val.shape[0], len(base_models)))

    for i, (name, model) in enumerate(base_models.items()):
        if hasattr(model, 'predict_proba'):
            val_preds = model.predict_proba(X_val)[:, 1]
        else:
            val_preds = model.predict(X_val)

        meta_preds_val[:, i] = val_preds

    return meta_features_train, meta_preds_val


def tune_meta_learner(meta_features_train, y_train, meta_preds_val, y_val, n_trials=50, n_jobs=1):
    """
    Tune meta-learner model with parallel processing
    """

    def objective(trial):
        model_type = trial.suggest_categorical('model_type', ['catboost', 'xgboost', 'logistic'])

        if model_type == 'catboost':
            params = {
                'iterations': trial.suggest_int('iterations', 100, 500),
                'depth': trial.suggest_int('depth', 3, 8),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.1, 10.0, log=True),
                'verbose': 0,
                'thread_count': n_jobs if n_jobs > 0 else None,
                'random_seed': 42
            }
            model = cb.CatBoostClassifier(**params)

        elif model_type == 'xgboost':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 300),
                'max_depth': trial.suggest_int('max_depth', 2, 7),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'n_jobs': n_jobs,
                'random_state': 42
            }
            model = xgb.XGBClassifier(**params)

        else:
            params = {
                'C': trial.suggest_float('C', 0.001, 10.0, log=True),
                'solver': trial.suggest_categorical('solver', ['liblinear', 'saga']),
                'max_iter': 1000,
                'n_jobs': n_jobs,
                'random_state': 42
            }
            model = LogisticRegression(**params)

        model.fit(meta_features_train, y_train)

        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(meta_preds_val)[:, 1]

            thresholds = np.linspace(0.3, 0.7, 20)
            best_acc = 0
            for threshold in thresholds:
                y_pred = (y_pred_proba > threshold).astype(int)
                acc = accuracy_score(y_val, y_pred)
                if acc > best_acc:
                    best_acc = acc

            return best_acc
        else:
            y_pred = model.predict(meta_preds_val)
            return accuracy_score(y_val, y_pred)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, timeout=600, n_jobs=n_jobs)

    print(f"Best meta-learner parameters: {study.best_params}")

    best_params = study.best_params.copy()
    model_type = best_params.pop('model_type')

    if model_type == 'catboost':
        best_params['verbose'] = 0
        best_params['thread_count'] = n_jobs if n_jobs > 0 else None
        best_params['random_seed'] = 42
        best_model = cb.CatBoostClassifier(**best_params)
    elif model_type == 'xgboost':
        best_params['n_jobs'] = n_jobs
        best_params['random_state'] = 42
        best_model = xgb.XGBClassifier(**best_params)
    else:  # logistic
        best_params['n_jobs'] = n_jobs
        best_params['max_iter'] = 1000
        best_params['random_state'] = 42
        best_model = LogisticRegression(**best_params)

    best_model.fit(meta_features_train, y_train)

    if hasattr(best_model, 'predict_proba'):
        y_pred_proba = best_model.predict_proba(meta_preds_val)[:, 1]

        thresholds = np.linspace(0.3, 0.7, 40)
        best_acc = 0
        best_threshold = 0.5

        for threshold in thresholds:
            y_pred = (y_pred_proba > threshold).astype(int)
            acc = accuracy_score(y_val, y_pred)
            if acc > best_acc:
                best_acc = acc
                best_threshold = threshold

        print(f"Best threshold: {best_threshold:.4f}")
        best_model.threshold = best_threshold
        best_score = best_acc
    else:
        best_score = accuracy_score(y_val, best_model.predict(meta_preds_val))

    print(f"Best meta-learner accuracy: {best_score:.4f}")
    return best_model, best_score

In [20]:
import os
import time

os.environ["OMP_NUM_THREADS"] = str(os.cpu_count())
os.environ["MKL_NUM_THREADS"] = str(os.cpu_count())
os.environ["VECLIB_MAXIMUM_THREADS"] = str(os.cpu_count())

start_time = time.time()


def run_optimized_spaceship_solution(n_jobs=-1):
    """
    Run the fully optimized spaceship Titanic solution with separate data formats for each model type
    """
    print("===== DATA PREPARATION =====")
    data_dict = prepare_spaceship_data(train_data, test_data)

    print("Applying enhanced feature engineering...")
    dfs = [
        (data_dict['X_train'], 'train'),
        (data_dict['X_val'], 'val'),
        (data_dict['X_test'], 'test')
    ]
    combined = pd.concat([df.assign(source=source) for df, source in dfs], axis=0)

    combined = enhanced_feature_engineering(combined)

    data_dict['X_train'] = combined[combined['source'] == 'train'].drop('source', axis=1)
    data_dict['X_val'] = combined[combined['source'] == 'val'].drop('source', axis=1)
    data_dict['X_test'] = combined[combined['source'] == 'test'].drop('source', axis=1)

    print("Removing individual name features to prevent dimensionality explosion...")
    name_columns = [col for col in data_dict['X_train'].columns if col.startswith(('FirstName_', 'LastName_'))]
    print(f"Removing {len(name_columns)} name-based features")

    for dataset_name in ['X_train', 'X_val', 'X_test']:
        data_dict[dataset_name] = data_dict[dataset_name].drop(columns=name_columns)

    print(f"Feature count after name removal: {data_dict['X_train'].shape[1]}")

    print("\n===== STRICT CATEGORICAL FEATURE SELECTION =====")
    preserved_cat_cols = []
    if 'preserved_categorical_cols' in data_dict:
        preserved_cat_cols = data_dict['preserved_categorical_cols']
        print(f"Found {len(preserved_cat_cols)} preserved categorical features")
    else:
        preserved_cat_cols = [col for col in data_dict['X_train'].columns if col.startswith('cat_')]
        print(f"Identified {len(preserved_cat_cols)} preserved categorical features by prefix")

    binary_cat_cols = []

    strict_binary_patterns = [
        'CryoSleep',
        'VIP',
        'PassengerGroup',
        'TravelingAlone',
        'HasFamily',
        'HomePlanet_',
        'Destination_',
        'CabinDeck_',
        'CabinSide_',
        'Route_'
    ]

    for col in data_dict['X_train'].columns:

        if any(pattern in col for pattern in strict_binary_patterns):
            binary_cat_cols.append(col)

    if len(binary_cat_cols) > 50:
        print(f"WARNING: Too many binary categorical features ({len(binary_cat_cols)}), limiting to 50 most important")
        binary_cat_cols = binary_cat_cols[:50]

    print(f"Selected {len(binary_cat_cols)} additional binary categorical features")

    categorical_cols = preserved_cat_cols + binary_cat_cols
    print(f"Using total of {len(categorical_cols)} categorical features for CatBoost")

    if preserved_cat_cols:
        print(f"Sample preserved features: {preserved_cat_cols[:5]}")
    if binary_cat_cols:
        print(f"Sample binary features: {binary_cat_cols[:5]}")

    numerical_cols = [col for col in data_dict['X_train'].columns if col not in categorical_cols]
    print(f"Identified {len(numerical_cols)} numerical features")

    print("\n===== FEATURE COUNT DIAGNOSTICS =====")
    print(f"Total features before selection: {data_dict['X_train'].shape[1]}")
    print(f"Number of preserved categorical features: {len(preserved_cat_cols)}")
    print(f"Number of binary categorical features: {len(binary_cat_cols)}")
    print(f"Number of numerical features: {len(numerical_cols)}")

    from sklearn.impute import SimpleImputer
    if numerical_cols:
        num_imputer = SimpleImputer(strategy='median')

        num_imputer.fit(data_dict['X_train'][numerical_cols])

        for dataset_name in ['X_train', 'X_val', 'X_test']:
            numerical_data = data_dict[dataset_name][numerical_cols]
            imputed_data = pd.DataFrame(
                num_imputer.transform(numerical_data),
                columns=numerical_cols,
                index=data_dict[dataset_name].index
            )

            for col in numerical_cols:
                data_dict[dataset_name][col] = imputed_data[col]

    if categorical_cols:
        cat_imputer = SimpleImputer(strategy='most_frequent')

        cat_imputer.fit(data_dict['X_train'][categorical_cols])

        for dataset_name in ['X_train', 'X_val', 'X_test']:
            categorical_data = data_dict[dataset_name][categorical_cols]
            imputed_data = pd.DataFrame(
                cat_imputer.transform(categorical_data),
                columns=categorical_cols,
                index=data_dict[dataset_name].index
            )

            for col in categorical_cols:
                data_dict[dataset_name][col] = imputed_data[col]

    print("\n===== CREATING MODEL-SPECIFIC DATASETS =====")

    data_dict['X_train_catboost'] = data_dict['X_train'].copy()
    data_dict['X_val_catboost'] = data_dict['X_val'].copy()
    data_dict['X_test_catboost'] = data_dict['X_test'].copy()

    print("Creating CatBoost dataset with string categorical features...")
    for col in categorical_cols:
        for dataset_name in ['X_train_catboost', 'X_val_catboost', 'X_test_catboost']:
            data_dict[dataset_name][col] = data_dict[dataset_name][col].astype(str)

    data_dict['X_train_xgb'] = data_dict['X_train'].copy()
    data_dict['X_val_xgb'] = data_dict['X_val'].copy()
    data_dict['X_test_xgb'] = data_dict['X_test'].copy()

    print("Creating XGBoost dataset with pandas category features...")
    for col in categorical_cols:
        for dataset_name in ['X_train_xgb', 'X_val_xgb', 'X_test_xgb']:
            data_dict[dataset_name][col] = data_dict[dataset_name][col].astype('category')

    data_dict['X_train_sklearn'] = data_dict['X_train'].copy()
    data_dict['X_val_sklearn'] = data_dict['X_val'].copy()
    data_dict['X_test_sklearn'] = data_dict['X_test'].copy()

    print("Creating sklearn dataset with numerical features only...")
    from sklearn.preprocessing import OrdinalEncoder

    cols_to_encode = []
    for col in data_dict['X_train_sklearn'].columns:
        if data_dict['X_train_sklearn'][col].dtype == 'object' or isinstance(data_dict['X_train_sklearn'][col].dtype,
                                                                             pd.CategoricalDtype):
            cols_to_encode.append(col)

    if cols_to_encode:
        print(f"Encoding {len(cols_to_encode)} categorical features for sklearn models")
        ord_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

        ord_encoder.fit(data_dict['X_train_sklearn'][cols_to_encode])

        for dataset_name in ['X_train_sklearn', 'X_val_sklearn', 'X_test_sklearn']:
            encoded_data = ord_encoder.transform(data_dict[dataset_name][cols_to_encode])

            for i, col in enumerate(cols_to_encode):
                data_dict[dataset_name][col] = encoded_data[:, i]

    print("\n===== OPTUNA HYPERPARAMETER OPTIMIZATION =====")
    import multiprocessing
    cpu_count = multiprocessing.cpu_count()
    print(f"System has {cpu_count} CPUs available")

    if n_jobs == -1:
        n_jobs = cpu_count

    print(f"Using {n_jobs} parallel jobs for optimization")

    data_dict['categorical_features'] = categorical_cols

    ensemble = tune_with_optuna_multi_data(
        data_dict,
        categorical_cols=categorical_cols,
        n_trials=100,
        n_jobs=n_jobs
    )

    print("\n===== FINAL PREDICTION =====")
    meta_features_test = np.zeros((data_dict['X_test'].shape[0], len(ensemble['base_models'])))

    for i, (name, model) in enumerate(ensemble['base_models'].items()):
        if name == 'catboost':
            meta_features_test[:, i] = model.predict_proba(data_dict['X_test_catboost'])[:, 1]
        elif name == 'xgboost':
            meta_features_test[:, i] = model.predict_proba(data_dict['X_test_xgb'])[:, 1]
        else:
            if hasattr(model, 'predict_proba'):
                meta_features_test[:, i] = model.predict_proba(data_dict['X_test_sklearn'])[:, 1]
            else:
                meta_features_test[:, i] = model.predict(data_dict['X_test_sklearn'])

    meta_learner = ensemble['meta_learner']
    if hasattr(meta_learner, 'predict_proba'):
        final_probs = meta_learner.predict_proba(meta_features_test)[:, 1]
        threshold = meta_learner.threshold if hasattr(meta_learner, 'threshold') else 0.5
        final_preds = (final_probs > threshold).astype(bool)
    else:
        final_preds = meta_learner.predict(meta_features_test).astype(bool)

    submission = pd.DataFrame({
        'PassengerId': data_dict['test_ids'],
        'Transported': final_preds
    })

    submission_path = 'optimized_submission.csv'
    submission.to_csv(submission_path, index=False)
    print(f"Optimized submission file created: {submission_path}")

    return {
        'ensemble': ensemble,
        'submission': submission,
        'data_dict': data_dict
    }

In [281]:
def create_weighted_ensemble(base_models, meta_features_train, meta_preds_val, y_train, y_val):
    """
    Create a weighted ensemble using the pre-calculated weights
    
    Parameters:
    -----------
    base_models : dict
        Dictionary of trained base models
    meta_features_train : numpy.ndarray
        Meta-features for training (predictions from base models)
    meta_preds_val : numpy.ndarray
        Meta-predictions for validation (predictions from base models)
    y_train : pandas.Series
        Training target values
    y_val : pandas.Series
        Validation target values
    
    Returns:
    --------
    dict : Ensemble dictionary with base models and meta-learner
    """
    print("\n==== Creating Weighted Ensemble ====")

    base_scores = np.array([0.8189, 0.8246, 0.7993, 0.8051, 0.7982])

    temperature = 5.0  # Higher = more focused on best models
    weights = np.exp(temperature * (base_scores - np.max(base_scores)))
    weights = weights / np.sum(weights)

    print(f"Model weights: {weights}")

    class WeightedEnsemble:
        def __init__(self, weights, threshold=0.5):
            self.weights = weights
            self.threshold = threshold

        def predict_proba(self, X):
            weighted_avg = np.average(X, axis=1, weights=self.weights)
            return np.column_stack([1 - weighted_avg, weighted_avg])

        def predict(self, X):
            probs = self.predict_proba(X)[:, 1]
            return (probs > self.threshold).astype(int)

    weighted_val_preds = np.average(meta_preds_val, axis=1, weights=weights)

    best_threshold = 0.5
    best_acc = 0
    from sklearn.metrics import accuracy_score

    for threshold in np.linspace(0.3, 0.7, 41):
        acc = accuracy_score(y_val, (weighted_val_preds > threshold).astype(int))
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold

    print(f"Best threshold: {best_threshold}, Weighted Ensemble Accuracy: {best_acc:.4f}")

    ensemble = {
        'base_models': base_models,
        'meta_learner': WeightedEnsemble(weights=weights, threshold=best_threshold)
    }

    return ensemble

In [ ]:
def tune_xgboost_focused(data_dict, n_trials=200, n_jobs=-1):
    """
    Focused Optuna tuning just for XGBoost with more trials and wider search space
    """
    print("\n==== Enhanced XGBoost Tuning ====")

    X_train = data_dict['X_train'].copy()
    y_train = data_dict['y_train']
    X_val = data_dict['X_val'].copy()
    y_val = data_dict['y_val']

    cat_cols = [col for col in X_train.columns if
                X_train[col].dtype == 'object' or
                col.startswith('cat_') or
                isinstance(X_train[col].dtype, pd.CategoricalDtype)]

    for col in cat_cols:
        X_train[col] = X_train[col].astype('category')
        X_val[col] = X_val[col].astype('category')

    print(f"Converted {len(cat_cols)} columns to category type")

    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0.01, 2.0, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 2.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 2.0, log=True),
            'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.8, 1.2),
            'tree_method': 'hist'
        }

        early_stopping_rounds = 50

        model = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss')
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=early_stopping_rounds,
            verbose=False
        )

        if hasattr(model, 'best_iteration'):
            preds = model.predict_proba(X_val, iteration_range=(0, model.best_iteration + 1))[:, 1]
        else:
            preds = model.predict_proba(X_val)[:, 1]

        thresholds = np.linspace(0.3, 0.7, 41)
        scores = []
        for threshold in thresholds:
            scores.append(accuracy_score(y_val, (preds > threshold).astype(int)))

        best_idx = np.argmax(scores)
        best_threshold = thresholds[best_idx]
        best_score = scores[best_idx]

        trial.set_user_attr('threshold', float(best_threshold))
        trial.set_user_attr('best_iteration', int(model.best_iteration) if hasattr(model, 'best_iteration') else None)

        return best_score

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs, show_progress_bar=True)

    best_params = study.best_params
    best_threshold = study.best_trial.user_attrs['threshold']
    best_iteration = study.best_trial.user_attrs.get('best_iteration')

    print(f"Best XGBoost parameters: {best_params}")
    if best_iteration:
        print(f"Best iteration: {best_iteration}")

    final_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
    final_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False,
        early_stopping_rounds=50
    )

    final_model.threshold = best_threshold

    if hasattr(final_model, 'best_iteration'):
        val_preds = final_model.predict_proba(X_val, iteration_range=(0, final_model.best_iteration + 1))[:, 1]
    else:
        val_preds = final_model.predict_proba(X_val)[:, 1]

    accuracy = accuracy_score(y_val, (val_preds > best_threshold).astype(int))
    print(f"Best accuracy: {accuracy:.4f} with threshold: {best_threshold:.4f}")

    X_test = data_dict['X_test'].copy()
    for col in cat_cols:
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')

    return final_model, accuracy

In [298]:
def run_xgboost_optimized_solution(n_jobs=-1, n_trials=200):
    """
    Run an optimized XGBoost solution using the same approach that works well in ensemble
    """
    print("===== DATA PREPARATION =====")
    data_dict = prepare_spaceship_data(train_data, test_data)

    print("Applying enhanced feature engineering...")
    dfs = [
        (data_dict['X_train'], 'train'),
        (data_dict['X_val'], 'val'),
        (data_dict['X_test'], 'test')
    ]
    combined = pd.concat([df.assign(source=source) for df, source in dfs], axis=0)

    combined = enhanced_feature_engineering(combined)

    data_dict['X_train'] = combined[combined['source'] == 'train'].drop('source', axis=1)
    data_dict['X_val'] = combined[combined['source'] == 'val'].drop('source', axis=1)
    data_dict['X_test'] = combined[combined['source'] == 'test'].drop('source', axis=1)

    print("Removing individual name features to prevent dimensionality explosion...")
    name_columns = [col for col in data_dict['X_train'].columns if col.startswith(('FirstName_', 'LastName_'))]
    print(f"Removing {len(name_columns)} name-based features")

    for dataset_name in ['X_train', 'X_val', 'X_test']:
        data_dict[dataset_name] = data_dict[dataset_name].drop(columns=name_columns)

    print(f"Feature count after name removal: {data_dict['X_train'].shape[1]}")

    print("\n===== CATEGORICAL FEATURE IDENTIFICATION =====")
    preserved_cat_cols = []
    if 'preserved_categorical_cols' in data_dict:
        preserved_cat_cols = data_dict['preserved_categorical_cols']
        print(f"Found {len(preserved_cat_cols)} preserved categorical features")
    else:
        preserved_cat_cols = [col for col in data_dict['X_train'].columns if col.startswith('cat_')]
        print(f"Identified {len(preserved_cat_cols)} preserved categorical features by prefix")

    binary_cat_cols = []
    strict_binary_patterns = [
        'CryoSleep',
        'VIP',
        'PassengerGroup',
        'TravelingAlone',
        'HasFamily',
        'HomePlanet_',
        'Destination_',
        'CabinDeck_',
        'CabinSide_',
        'Route_'
    ]

    for col in data_dict['X_train'].columns:
        if any(pattern in col for pattern in strict_binary_patterns):
            binary_cat_cols.append(col)

    if len(binary_cat_cols) > 50:
        print(f"WARNING: Too many binary categorical features ({len(binary_cat_cols)}), limiting to 50 most important")
        binary_cat_cols = binary_cat_cols[:50]

    print(f"Selected {len(binary_cat_cols)} additional binary categorical features")

    categorical_cols = preserved_cat_cols + binary_cat_cols
    print(f"Using total of {len(categorical_cols)} categorical features")

    numerical_cols = [col for col in data_dict['X_train'].columns if col not in categorical_cols]
    print(f"Identified {len(numerical_cols)} numerical features")

    from sklearn.impute import SimpleImputer
    if numerical_cols:
        num_imputer = SimpleImputer(strategy='median')
        num_imputer.fit(data_dict['X_train'][numerical_cols])

        for dataset_name in ['X_train', 'X_val', 'X_test']:
            numerical_data = data_dict[dataset_name][numerical_cols]
            imputed_data = pd.DataFrame(
                num_imputer.transform(numerical_data),
                columns=numerical_cols,
                index=data_dict[dataset_name].index
            )

            for col in numerical_cols:
                data_dict[dataset_name][col] = imputed_data[col]

    if categorical_cols:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        cat_imputer.fit(data_dict['X_train'][categorical_cols])

        for dataset_name in ['X_train', 'X_val', 'X_test']:
            categorical_data = data_dict[dataset_name][categorical_cols]
            imputed_data = pd.DataFrame(
                cat_imputer.transform(categorical_data),
                columns=categorical_cols,
                index=data_dict[dataset_name].index
            )

            for col in categorical_cols:
                data_dict[dataset_name][col] = imputed_data[col]

    print("\n===== PREPARING XGBOOST-SPECIFIC DATASET =====")
    X_train_xgb = data_dict['X_train'].copy()
    X_val_xgb = data_dict['X_val'].copy()
    X_test_xgb = data_dict['X_test'].copy()

    print("Converting categorical features to category type...")
    for col in categorical_cols:
        X_train_xgb[col] = X_train_xgb[col].astype('category')
        X_val_xgb[col] = X_val_xgb[col].astype('category')
        X_test_xgb[col] = X_test_xgb[col].astype('category')

    print(f"Converted {len(categorical_cols)} columns to category type")

    print("\n===== OPTUNA HYPERPARAMETER OPTIMIZATION FOR XGBOOST =====")
    import multiprocessing
    import optuna
    import xgboost as xgb
    from sklearn.metrics import accuracy_score
    import numpy as np

    cpu_count = multiprocessing.cpu_count()
    print(f"System has {cpu_count} CPUs available")

    if n_jobs == -1:
        n_jobs = cpu_count

    print(f"Using {n_jobs} parallel jobs for optimization")

    def objective(trial):

        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0.01, 1.0, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10.0, log=True),
            'n_jobs': n_jobs,
            'enable_categorical': True
        }

        model = xgb.XGBClassifier(**params, random_state=42)

        try:

            callbacks = [xgb.callback.EarlyStopping(rounds=50, verbose=False)]
            model.fit(X_train_xgb, data_dict['y_train'],
                      eval_set=[(X_val_xgb, data_dict['y_val'])],
                      callbacks=callbacks, verbose=False)
        except:
            try:

                model.fit(X_train_xgb, data_dict['y_train'],
                          eval_set=[(X_val_xgb, data_dict['y_val'])],
                          early_stopping_rounds=50, verbose=False)
            except:

                model.fit(X_train_xgb, data_dict['y_train'])

        preds = model.predict_proba(X_val_xgb)[:, 1]
        thresholds = np.linspace(0.4, 0.6, 20)
        best_acc = 0
        best_thresh = 0.5

        for threshold in thresholds:
            binary_preds = (preds > threshold).astype(int)
            acc = accuracy_score(data_dict['y_val'], binary_preds)
            if acc > best_acc:
                best_acc = acc
                best_thresh = threshold

        trial.set_user_attr('threshold', float(best_thresh))

        return best_acc

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs)

    best_params = study.best_params
    best_threshold = study.best_trial.user_attrs['threshold']

    print(f"Best XGBoost parameters: {best_params}")
    print(f"Best threshold: {best_threshold:.4f}")

    print("\n===== TRAINING FINAL MODEL =====")
    best_params['n_jobs'] = n_jobs
    best_params['enable_categorical'] = True

    final_model = xgb.XGBClassifier(**best_params, random_state=42)

    try:
        callbacks = [xgb.callback.EarlyStopping(rounds=50, verbose=False)]
        final_model.fit(X_train_xgb, data_dict['y_train'],
                        eval_set=[(X_val_xgb, data_dict['y_val'])],
                        callbacks=callbacks, verbose=False)
    except:
        try:

            final_model.fit(X_train_xgb, data_dict['y_train'],
                            eval_set=[(X_val_xgb, data_dict['y_val'])],
                            early_stopping_rounds=50, verbose=False)
        except:

            final_model.fit(X_train_xgb, data_dict['y_train'])

    final_model.threshold = best_threshold

    val_preds = final_model.predict_proba(X_val_xgb)[:, 1]
    val_accuracy = accuracy_score(data_dict['y_val'], (val_preds > best_threshold).astype(int))
    print(f"Validation accuracy: {val_accuracy:.4f} with threshold: {best_threshold:.4f}")

    print("\n===== FINAL PREDICTION =====")
    test_preds = final_model.predict_proba(X_test_xgb)[:, 1]

    final_preds = (test_preds > best_threshold).astype(bool)

    submission = pd.DataFrame({
        'PassengerId': data_dict['test_ids'],
        'Transported': final_preds
    })

    submission_path = 'xgboost_optimized_submission.csv'
    submission.to_csv(submission_path, index=False)
    print(f"XGBoost submission file created: {submission_path}")
    print(f"Positive predictions: {submission['Transported'].sum()} ({submission['Transported'].mean() * 100:.2f}%)")

    return {
        'model': final_model,
        'submission': submission,
        'data_dict': data_dict,
        'validation_accuracy': val_accuracy
    }

In [285]:
def create_xgboost_submission(xgb_model, data_dict):
    """
    Create a submission file using the trained XGBoost model
    
    Parameters:
    -----------
    xgb_model : XGBClassifier
        Trained XGBoost model with threshold attribute
    data_dict : dict
        Dictionary containing data, including test set and passenger IDs
    
    Returns:
    --------
    pandas.DataFrame
        Submission dataframe with PassengerId and Transported columns
    """
    import pandas as pd
    import numpy as np

    X_test = data_dict['X_test'].copy()

    passenger_ids = data_dict['test_passenger_ids']

    cat_cols = [col for col in X_test.columns if
                X_test[col].dtype == 'object' or
                col.startswith('cat_') or
                isinstance(X_test[col].dtype, pd.CategoricalDtype)]

    for col in cat_cols:
        X_test[col] = X_test[col].astype('category')

    if hasattr(xgb_model, 'best_iteration') and xgb_model.best_iteration is not None:
        test_preds = xgb_model.predict_proba(X_test, iteration_range=(0, xgb_model.best_iteration + 1))[:, 1]
    else:
        test_preds = xgb_model.predict_proba(X_test)[:, 1]

    threshold = xgb_model.threshold if hasattr(xgb_model, 'threshold') else 0.5
    predicted_labels = (test_preds > threshold).astype(bool)

    submission = pd.DataFrame({
        'PassengerId': passenger_ids,
        'Transported': predicted_labels
    })

    print(f"Created submission with {len(submission)} rows")
    print(f"Positive predictions: {submission['Transported'].sum()} ({submission['Transported'].mean() * 100:.2f}%)")

    return submission

In [255]:
def adapt_data_for_catboost(X_train, y_train, X_val, y_val, X_test, test_ids):
    """
    Creates a CatBoost-friendly version of the data while preserving the original processed data
    for other models in the ensemble.
    
    This function works with your existing processed data and recovers categorical features.
    """
    print("\n===== ADAPTING DATA FOR CATBOOST =====")

    categorical_groups = {}

    categorical_prefixes = [
        'HomePlanet_', 'Destination_', 'CabinDeck_', 'CabinSide_',
        'AgeGroup_', 'Route_', 'LastName_', 'FirstName_'
    ]

    standard_categorical = ['CryoSleep', 'VIP', 'PassengerGroup']

    all_columns = list(X_train.columns)
    for prefix in categorical_prefixes:
        matching_cols = [col for col in all_columns if col.startswith(prefix)]
        if matching_cols:
            original_feature = prefix.rstrip('_')
            categorical_groups[original_feature] = matching_cols

    for col in standard_categorical:
        if col in all_columns:
            categorical_groups[col] = [col]

    print(f"Identified {len(categorical_groups)} categorical feature groups")

    cb_X_train = X_train.copy()
    cb_X_val = X_val.copy()
    cb_X_test = X_test.copy()

    all_categorical_columns = []
    for original_feature, cols in categorical_groups.items():
        if len(cols) > 1:

            for df_name, df in [('train', cb_X_train), ('val', cb_X_val), ('test', cb_X_test)]:
                df[original_feature] = "Other"

                for col in cols:
                    category = col[len(original_feature) + 1:]

                    highest_value_mask = df[col] > df[cols].mean(axis=1)
                    df.loc[highest_value_mask, original_feature] = category

                df.drop(cols, axis=1, inplace=True)

            all_categorical_columns.append(original_feature)
        else:

            all_categorical_columns.append(cols[0])

    for col in all_categorical_columns:
        if col in cb_X_train.columns:
            for df in [cb_X_train, cb_X_val, cb_X_test]:
                df[col] = df[col].astype(str)

    print(f"Created {len(all_categorical_columns)} categorical columns for CatBoost")

    return {
        'cb_X_train': cb_X_train,
        'cb_X_val': cb_X_val,
        'cb_X_test': cb_X_test,
        'categorical_features': all_categorical_columns,
        'y_train': y_train,
        'y_val': y_val,
        'test_ids': test_ids
    }


def prepare_hybrid_catboost_data(data_dict):
    """
    Create a hybrid version with both preserved categorical features and original binary features
    """
    print("Creating hybrid CatBoost data with both preserved and binary features")

    X_train_hybrid = data_dict['X_train'].copy()
    X_val_hybrid = data_dict['X_val'].copy()
    X_test_hybrid = data_dict['X_test'].copy()

    binary_cat_cols = [
        col for col in data_dict['X_train'].columns
        if col.startswith(('FirstName_', 'LastName_', 'CryoSleep', 'VIP', 'PassengerGroup'))
    ]

    preserved_cat_cols = []
    if 'preserved_categorical_cols' in data_dict:
        preserved_cat_cols = data_dict['preserved_categorical_cols']

    all_cat_cols = binary_cat_cols + preserved_cat_cols
    print(f"Using {len(preserved_cat_cols)} preserved and {len(binary_cat_cols)} binary categorical features")

    return {
        'X_train': X_train_hybrid,
        'X_val': X_val_hybrid,
        'X_test': X_test_hybrid,
        'categorical_features': all_cat_cols
    }

In [283]:
run_optimized_spaceship_solution()

===== DATA PREPARATION =====
Loading and preparing Spaceship Titanic data with preserved categorical features...
Training data: 8693 rows, 14 columns
Test data: 4277 rows, 13 columns
Converted boolean target to integer (0/1)
Split into 6954 training samples and 1739 validation samples


/var/folders/4k/r28hjx957zd3v1xzgv0q81lr0000gn/T/ipykernel_47402/3400320643.py:96: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0. 0. 0. ... 0. 0. 0.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/4k/r28hjx957zd3v1xzgv0q81lr0000gn/T/ipykernel_47402/3400320643.py:96: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.         0.2699005  0.99688958 ... 0.99654378 0.83202358 0.83671558]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/4k/r28hjx957zd3v1xzgv0q81lr0000gn/T/ipykernel_47402/3400320643.py:96: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.01152074 0.         0.         ... 0.         0.01669941 0.        ]' has dtype inco

Preserving categorical feature: HomePlanet
Preserving categorical feature: Destination
Preserving categorical feature: CabinDeck
Preserving categorical feature: CabinSide
Preserving categorical feature: AgeGroup
Preserving categorical feature: Route
After preprocessing: 5350 features created
Added 6 preserved categorical features
Found NaN values in X_train_proc for columns: ['CabinNum', 'FamilySize', 'cat_AgeGroup']
Filling remaining NaNs with appropriate values...
  - CabinNum: filled with median (429.0)
  - FamilySize: filled with median (6.0)
  - cat_AgeGroup: filled with mode (Adult)
Found NaN values in X_val_proc for columns: ['CabinNum', 'FamilySize', 'cat_AgeGroup']
Filling remaining NaNs with appropriate values...
  - CabinNum: filled with median (420.0)
  - FamilySize: filled with median (6.0)
  - cat_AgeGroup: filled with mode (Adult)
Found NaN values in X_test_proc for columns: ['CabinNum', 'FamilySize', 'cat_AgeGroup']
Filling remaining NaNs with appropriate values...
  - 

{'ensemble': {'base_models': {'catboost': <catboost.core.CatBoostClassifier at 0x6b9cfbd60>,
   'xgboost': XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=0.9651850149911293, device=None,
                 early_stopping_rounds=None, enable_categorical=True,
                 eval_metric=None, feature_types=None, gamma=0.23511026951869188,
                 grow_policy=None, importance_type=None,
                 interaction_constraints=None, learning_rate=0.028277968827625385,
                 max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
                 max_delta_step=None, max_depth=6, max_leaves=None,
                 min_child_weight=8, missing=nan, monotone_constraints=None,
                 multi_strategy=None, n_estimators=794, n_jobs=11,
                 num_parallel_tree=None, random_state=42, ...),
   'random_forest': RandomForestClassifier(max_depth=3

In [12]:
def elite_xgboost_solution(train_data, test_data, random_state=42, n_trials=50):
    import pandas as pd
    import numpy as np
    import xgboost as xgb
    from sklearn.model_selection import train_test_split, StratifiedKFold
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.metrics import accuracy_score
    import time
    import optuna
    import warnings
    warnings.filterwarnings('ignore')

    start_time = time.time()
    print("===== XGBOOST SOLUTION FOR SPACESHIP TITANIC =====")

    print("\n[1/8] Data Preparation")

    test_ids = test_data['PassengerId'].copy()

    y = train_data['Transported'].copy()
    if y.dtype == bool:
        y = y.astype(int)
        print("Converted boolean target to integer (0/1)")

    X = train_data.drop(['Transported'], axis=1)

    df_train = train_data.copy()
    df_test = test_data.copy()
    all_data = pd.concat([X, df_test], axis=0).reset_index(drop=True)

    print("[2/8] Basic Feature Engineering")

    if 'PassengerId' in all_data.columns:
        all_data['PassengerGroup'] = all_data['PassengerId'].str.split('_').str[0].astype(int)
        all_data['PassengerNumber'] = all_data['PassengerId'].str.split('_').str[1].astype(int)

    if 'Cabin' in all_data.columns:
        all_data['HasCabin'] = all_data['Cabin'].notna().astype(int)

        all_data['Deck'] = 'Z'
        all_data['CabinNum'] = -1
        all_data['Side'] = 'Z'

        cabin_mask = all_data['Cabin'].notna()
        if cabin_mask.any():
            cabin_parts = all_data.loc[cabin_mask, 'Cabin'].str.split('/', expand=True)
            if cabin_parts.shape[1] >= 3:
                all_data.loc[cabin_mask, 'Deck'] = cabin_parts[0]
                all_data.loc[cabin_mask, 'CabinNum'] = pd.to_numeric(cabin_parts[1], errors='coerce')
                all_data.loc[cabin_mask, 'Side'] = cabin_parts[2]

        all_data['CabinPosition'] = all_data['CabinNum'] % 2
        all_data['Deck_is_ABC'] = all_data['Deck'].isin(['A', 'B', 'C']).astype(int)
        all_data['Deck_is_FG'] = all_data['Deck'].isin(['F', 'G']).astype(int)

    expense_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    available_expenses = [col for col in expense_columns if col in all_data.columns]

    for col in available_expenses:
        all_data[col] = all_data[col].fillna(0)

    if available_expenses:
        all_data['TotalExpense'] = all_data[available_expenses].sum(axis=1)

        for col in available_expenses:
            all_data[f'{col}_Ratio'] = all_data.apply(
                lambda x: x[col] / x['TotalExpense'] if x['TotalExpense'] > 0 else 0,
                axis=1
            )

        all_data['HasExpense'] = (all_data['TotalExpense'] > 0).astype(int)
        all_data['ExpenseCount'] = (all_data[available_expenses] > 0).sum(axis=1)

        all_data['ExpensePattern'] = all_data.apply(
            lambda x: ''.join(['1' if x[col] > 0 else '0' for col in available_expenses]),
            axis=1
        )

        all_data['LogExpense'] = np.log1p(all_data['TotalExpense'])
        all_data['SqrtExpense'] = np.sqrt(all_data['TotalExpense'])

    if 'PassengerGroup' in all_data.columns:
        group_sizes = all_data.groupby('PassengerGroup').size()
        all_data['GroupSize'] = all_data['PassengerGroup'].map(group_sizes)

        if 'TotalExpense' in all_data.columns:
            group_expenses = all_data.groupby('PassengerGroup')['TotalExpense'].agg(['sum', 'mean', 'std']).fillna(0)
            all_data['GroupTotalExpense'] = all_data['PassengerGroup'].map(group_expenses['sum'])
            all_data['GroupMeanExpense'] = all_data['PassengerGroup'].map(group_expenses['mean'])

            all_data['ExpenseVsGroup'] = all_data.apply(
                lambda x: x['TotalExpense'] / x['GroupMeanExpense'] if x['GroupMeanExpense'] > 0 else 0,
                axis=1
            )

    if 'Name' in all_data.columns:
        all_data['LastName'] = all_data['Name'].str.split(' ').str[0]

        family_sizes = all_data.groupby('LastName').size()
        all_data['FamilySize'] = all_data['LastName'].map(family_sizes)

    if 'GroupSize' in all_data.columns:
        all_data['IsAlone'] = (all_data['GroupSize'] == 1).astype(int)

    if 'FamilySize' in all_data.columns:
        all_data['HasFamily'] = (all_data['FamilySize'] > 1).astype(int)

    print("[2/6] Advanced Feature Encoding & Preprocessing")

    train_rows = len(X)
    train_with_target = all_data.iloc[:train_rows].copy()
    train_with_target['Transported'] = y

    mean_target_features = ['PassengerGroup', 'LastName', 'ExpensePattern']
    mean_target_features = [col for col in mean_target_features if col in all_data.columns]

    for col in mean_target_features:
        means = train_with_target.groupby(col)['Transported'].mean().to_dict()
        all_data[f'{col}_encoded'] = all_data[col].map(means).fillna(0.5)

    categorical_cols = ['HomePlanet', 'Destination', 'Deck', 'Side', 'DeckSide', 'Route']
    categorical_cols = [col for col in categorical_cols if col in all_data.columns]

    for col in categorical_cols:
        top_categories = all_data[col].value_counts().head(5).index.tolist()
        for category in top_categories:
            all_data[f'{col}_{category}'] = (all_data[col] == category).astype(int)

    for col in all_data.columns:
        if all_data[col].dtype == 'object' or all_data[col].dtype.name == 'category':
            le.fit(all_data[col].fillna('Unknown').astype(str))
            all_data[col + '_encoded'] = le.transform(all_data[col].fillna('Unknown').astype(str))

            if col + '_encoded' in all_data.columns and col != 'LastName':
                all_data = all_data.drop(columns=[col])

    print("[3/8] Advanced Feature Engineering")

    if 'HomePlanet' in all_data.columns:
        all_data['HomePlanet'] = all_data['HomePlanet'].fillna(all_data['HomePlanet'].mode()[0])

    if 'CryoSleep' in all_data.columns:
        all_data['CryoSleep'] = all_data['CryoSleep'].fillna(False)

    if 'Destination' in all_data.columns:
        all_data['Destination'] = all_data['Destination'].fillna(all_data['Destination'].mode()[0])

    if 'VIP' in all_data.columns:
        all_data['VIP'] = all_data['VIP'].fillna(False)

    if 'Age' in all_data.columns:
        all_data['Age'] = all_data['Age'].fillna(all_data['Age'].median())
        all_data['AgeBand'] = pd.qcut(all_data['Age'], 5, labels=False)
        all_data['Age_squared'] = all_data['Age'] ** 2
        all_data['IsChild'] = (all_data['Age'] < 13).astype(int)
        all_data['IsAdult'] = ((all_data['Age'] >= 13) & (all_data['Age'] < 65)).astype(int)
        all_data['IsSenior'] = (all_data['Age'] >= 65).astype(int)

        if 'HomePlanet' in all_data.columns:
            planets = all_data['HomePlanet'].unique()
            for planet in planets:
                all_data[f'Age_from_{planet}'] = all_data['Age'] * (all_data['HomePlanet'] == planet)

    if all(col in all_data.columns for col in ['CryoSleep', 'VIP']):
        all_data['CryoSleep_VIP'] = (all_data['CryoSleep'] & all_data['VIP']).astype(int)

    if all(col in all_data.columns for col in ['CryoSleep', 'TotalExpense']):
        all_data['CryoSleep_HasExpense'] = (all_data['CryoSleep'] & (all_data['TotalExpense'] > 0)).astype(int)
        all_data['Unusual_Expense'] = ((all_data['CryoSleep'] & (all_data['TotalExpense'] > 0)) |
                                       (~all_data['CryoSleep'] & (all_data['TotalExpense'] == 0))).astype(int)

    if all(col in all_data.columns for col in ['HomePlanet', 'Destination']):
        all_data['Route'] = all_data['HomePlanet'] + '_to_' + all_data['Destination']

        route_counts = all_data['Route'].value_counts()
        all_data['RoutePopularity'] = all_data['Route'].map(route_counts)

    if all(col in all_data.columns for col in ['Deck', 'Side']):
        all_data['DeckSide'] = all_data['Deck'] + all_data['Side']

    if all(col in all_data.columns for col in ['VIP', 'TotalExpense']):
        all_data['Expense_by_VIP'] = all_data['TotalExpense'] * all_data['VIP'].astype(int)

    if all(col in all_data.columns for col in ['Age', 'TotalExpense']):
        all_data['ExpenseRatio_by_Age'] = all_data['TotalExpense'] / (all_data['Age'] + 1)
    print("[4/8] Missing Value Handling and Clustering")

    cols_with_missing = all_data.columns[all_data.isnull().any()].tolist()
    for col in cols_with_missing:
        all_data[f'{col}_Missing'] = all_data[col].isnull().astype(int)

    numeric_cols_with_missing = [
        col for col in all_data.select_dtypes(include=['float64', 'int64']).columns
        if all_data[col].isnull().sum() > 0
    ]

    if numeric_cols_with_missing:
        mice_imputer = IterativeImputer(max_iter=10, random_state=random_state)
        all_data[numeric_cols_with_missing] = mice_imputer.fit_transform(all_data[numeric_cols_with_missing])

    numeric_cols_for_clustering = all_data.select_dtypes(include=['float64', 'int64']).columns.tolist()
    if len(numeric_cols_for_clustering) > 5:
        clustering_cols = ['Age', 'TotalExpense', 'GroupSize', 'CabinNum']
        clustering_cols = [col for col in clustering_cols if col in numeric_cols_for_clustering]

        if len(clustering_cols) >= 3:
            kmeans = KMeans(n_clusters=5, random_state=random_state, n_init=10)
            all_data['Cluster'] = kmeans.fit_predict(all_data[clustering_cols].fillna(0))

    train_rows = len(X)
    train_with_target = all_data.iloc[:train_rows].copy()
    train_with_target['Transported'] = y

    mean_target_features = ['PassengerGroup', 'LastName', 'ExpensePattern', 'Deck', 'HomePlanet', 'Route']
    mean_target_features = [col for col in mean_target_features if col in all_data.columns]

    for col in mean_target_features:
        means = train_with_target.groupby(col)['Transported'].mean().to_dict()
        all_data[f'{col}_encoded'] = all_data[col].map(means).fillna(0.5)

    drop_cols = ['Name', 'Cabin', 'PassengerId']
    drop_cols = [col for col in drop_cols if col in all_data.columns]
    all_data = all_data.drop(columns=drop_cols)

    print("[5/8] Preprocessing and Encoding")

    categorical_cols = ['HomePlanet', 'Destination', 'Deck', 'Side', 'DeckSide', 'Route']
    categorical_cols = [col for col in categorical_cols if col in all_data.columns]

    for col in categorical_cols:
        top_categories = all_data[col].value_counts().head(5).index.tolist()
        for category in top_categories:
            all_data[f'{col}_{category}'] = (all_data[col] == category).astype(int)

    le = LabelEncoder()
    for col in all_data.columns:
        if all_data[col].dtype == 'object' or all_data[col].dtype.name == 'category':
            le.fit(all_data[col].fillna('Unknown').astype(str))
            all_data[col + '_encoded'] = le.transform(all_data[col].fillna('Unknown').astype(str))

            if col + '_encoded' in all_data.columns and col != 'LastName':
                all_data = all_data.drop(columns=[col])

    X_processed = all_data.iloc[:train_rows]
    X_test = all_data.iloc[train_rows:]

    X_train, X_val, y_train, y_val = train_test_split(
        X_processed,
        y,
        test_size=0.2,
        random_state=random_state,
        stratify=y
    )

    scaler = StandardScaler()
    numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()

    scaler.fit(X_train[numerical_cols])

    for df in [X_train, X_val, X_test]:
        df[numerical_cols] = scaler.transform(df[numerical_cols])

    for df in [X_train, X_val, X_test]:
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].astype('category').cat.codes

    print("[6/8] Optuna Hyperparameter Optimization")

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    def objective(trial):
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'eta': trial.suggest_float('eta', 0.01, 0.3),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 0, 5),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 5.0),
            'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.8, 1.2),
            'max_delta_step': trial.suggest_int('max_delta_step', 0, 5),
            'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
            'tree_method': 'hist',
        }

        n_estimators = trial.suggest_int('n_estimators', 300, 1000)

        cv_scores = []

        for train_idx, val_idx in cv.split(X_train, y_train):
            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            dtrain = xgb.DMatrix(X_fold_train, label=y_fold_train)
            dval = xgb.DMatrix(X_fold_val, label=y_fold_val)

            model = xgb.train(
                params,
                dtrain,
                num_boost_round=n_estimators,
                evals=[(dval, 'val')],
                early_stopping_rounds=50,
                verbose_eval=False
            )

            fold_preds = model.predict(dval)

            thresholds = np.linspace(0.3, 0.7, 21)
            scores = [accuracy_score(y_fold_val, (fold_preds > t).astype(int)) for t in thresholds]
            best_score = max(scores)
            cv_scores.append(best_score)

        return np.mean(cv_scores)

    print(f"Running {n_trials} Optuna trials...")
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    best_params = study.best_params.copy()
    best_params['objective'] = 'binary:logistic'
    best_params['eval_metric'] = 'logloss'
    best_params['tree_method'] = 'hist'
    n_estimators = best_params.pop('n_estimators')

    print(f"Best CV accuracy: {study.best_value:.4f}")
    print(f"Best parameters: {best_params}")
    print(f"Best n_estimators: {n_estimators}")

    print("[7/8] Model Training & Feature Selection")

    for df in [X_train, X_val, X_test]:
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].astype('category').cat.codes

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    cv_models = []
    cv_scores = []
    oof_predictions = np.zeros(len(X_train))
    feature_importances = pd.DataFrame()
    feature_importances['Feature'] = X_train.columns
    feature_importances['Importance'] = 0

    fold_idx = 0
    for train_idx, val_idx in cv.split(X_train, y_train):
        fold_idx += 1
        print(f"  Training fold {fold_idx}/5...")

        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        dtrain = xgb.DMatrix(X_fold_train, label=y_fold_train)
        dval = xgb.DMatrix(X_fold_val, label=y_fold_val)

        model = xgb.train(
            best_params,
            dtrain,
            num_boost_round=2000,
            evals=[(dval, 'val')],
            early_stopping_rounds=50,
            verbose_eval=False
        )
        importance = model.get_score(importance_type='gain')
        for feature, score in importance.items():
            if feature in feature_importances['Feature'].values:
                feature_importances.loc[feature_importances['Feature'] == feature, 'Importance'] += score

        fold_preds = model.predict(xgb.DMatrix(X_fold_val))
        oof_predictions[val_idx] = fold_preds

        best_threshold = 0.5
        thresholds = np.linspace(0.3, 0.7, 41)
        scores = [accuracy_score(y_fold_val, (fold_preds > t).astype(int)) for t in thresholds]
        if max(scores) > best_threshold:
            best_threshold = thresholds[np.argmax(scores)]

        fold_score = accuracy_score(y_fold_val, (fold_preds > best_threshold).astype(int))
        cv_scores.append(fold_score)

        cv_models.append((model, best_threshold))
        print(f"    Fold score: {fold_score:.4f}, Threshold: {best_threshold:.4f}")

    cv_mean = np.mean(cv_scores)
    cv_std = np.std(cv_scores)
    print(f"Cross-validation score: {cv_mean:.4f} ± {cv_std:.4f}")

    feature_importances = feature_importances.sort_values('Importance', ascending=False).reset_index(drop=True)
    important_features = feature_importances.iloc[:int(len(feature_importances) * 0.8)]['Feature'].tolist()

    print(f"Selected {len(important_features)} out of {len(X_train.columns)} features")

    X_train_selected = X_train[important_features]
    X_val_selected = X_val[important_features]
    X_test_selected = X_test[important_features]

    dtrain_selected = xgb.DMatrix(X_train_selected, label=y_train)
    dval_selected = xgb.DMatrix(X_val_selected, label=y_val)

    final_model = xgb.train(
        best_params,
        dtrain_selected,
        num_boost_round=n_estimators,
        evals=[(dval_selected, 'val')],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    print("[8/8] Advanced Ensemble Prediction & Submission")

    X_test_meta = np.zeros((len(X_test), len(cv_models) + 1))
    X_val_meta = np.zeros((len(X_val), len(cv_models) + 1))

    for i, (model, _) in enumerate(cv_models):
        X_test_meta[:, i] = model.predict(xgb.DMatrix(X_test))
        X_val_meta[:, i] = model.predict(xgb.DMatrix(X_val))

    X_test_meta[:, -1] = final_model.predict(xgb.DMatrix(X_test_selected))
    X_val_meta[:, -1] = final_model.predict(xgb.DMatrix(X_val_selected))

    meta_model = LogisticRegression(C=0.1, solver='liblinear', random_state=random_state)
    meta_model.fit(X_val_meta, y_val)

    test_meta_preds = meta_model.predict_proba(X_test_meta)[:, 1]

    thresholds = np.linspace(0.3, 0.7, 41)
    val_scores = [accuracy_score(y_val, (meta_model.predict_proba(X_val_meta)[:, 1] > t).astype(int))
                  for t in thresholds]
    best_threshold = thresholds[np.argmax(val_scores)]
    best_val_score = max(val_scores)
    print(f"Best stacking threshold: {best_threshold:.4f}, Validation Score: {best_val_score:.4f}")

    final_predictions = (test_meta_preds > best_threshold).astype(bool)

    submission = pd.DataFrame({
        'PassengerId': test_ids,
        'Transported': final_predictions
    })

    submission_path = 'enhanced_xgboost_submission.csv'
    submission.to_csv(submission_path, index=False)

    transport_rate = submission['Transported'].mean() * 100
    print(f"Prediction transport rate: {transport_rate:.2f}%")

    exec_time = time.time() - start_time
    print(f"\nExecution complete in {exec_time:.2f} seconds ({exec_time / 60:.2f} minutes)")

    return {
        'submission': submission,
        'cv_score': cv_mean,
        'validation_accuracy': best_val_score,
        'models': cv_models,
        'meta_model': meta_model,
        'best_threshold': best_threshold,
        'feature_importance': feature_importances
    }


def get_feature_importance(models, feature_names):
    importance_df = pd.DataFrame()
    importance_df['Feature'] = feature_names
    importance_df['Importance'] = 0
    for model, _ in models:
        importance = model.get_score(importance_type='gain')

        model_importance = pd.DataFrame({
            'Feature': list(importance.keys()),
            'Importance': list(importance.values())
        })

        importance_df = importance_df.merge(
            model_importance,
            on='Feature',
            how='left',
            suffixes=('_old', '')
        )

        importance_df['Importance'] = importance_df['Importance_old'].fillna(0) + importance_df['Importance'].fillna(0)

        importance_df = importance_df[['Feature', 'Importance']]

    importance_df['Importance'] /= len(models)

    return importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

In [ ]:
result = elite_xgboost_solution(train_data, test_data)

print(f"Validation accuracy: {result['cv_score']:.4f}")

print("\nTop 15 features:")
print(result['feature_importance'].head(15))
print(result['feature_importance'].head(15))

In [26]:
run_optimized_spaceship_solution()

===== DATA PREPARATION =====
Loading and preparing Spaceship Titanic data with preserved categorical features...
Training data: 8693 rows, 14 columns
Test data: 4277 rows, 13 columns
Converted boolean target to integer (0/1)
Split into 6954 training samples and 1739 validation samples
Preserving categorical feature: HomePlanet
Preserving categorical feature: Destination
Preserving categorical feature: CabinDeck
Preserving categorical feature: CabinSide
Preserving categorical feature: AgeGroup
Preserving categorical feature: Route
After preprocessing: 5350 features created
Added 6 preserved categorical features
Found NaN values in X_train_proc for columns: ['CabinNum', 'FamilySize', 'cat_AgeGroup']
Filling remaining NaNs with appropriate values...
  - CabinNum: filled with median (429.0)
  - FamilySize: filled with median (6.0)
  - cat_AgeGroup: filled with mode (Adult)
Found NaN values in X_val_proc for columns: ['CabinNum', 'FamilySize', 'cat_AgeGroup']
Filling remaining NaNs with app

[I 2025-03-18 01:01:59,120] A new study created in memory with name: no-name-8f025d8f-4e15-4894-bf27-b0ffa9a15707



===== OPTUNA HYPERPARAMETER OPTIMIZATION =====
System has 11 CPUs available
Using 11 parallel jobs for optimization
Starting comprehensive Optuna tuning with model-specific data formats...
Creating model-specific datasets...
Created specialized datasets for each model type
First 5 categorical features: ['cat_HomePlanet', 'cat_Destination', 'cat_CabinDeck', 'cat_CabinSide', 'cat_AgeGroup']
Tuning CatBoost with 34 categorical features
First 5 categorical features: ['cat_HomePlanet', 'cat_Destination', 'cat_CabinDeck', 'cat_CabinSide', 'cat_AgeGroup']


[I 2025-03-18 01:02:08,371] Trial 6 finished with value: 0.8027602070155262 and parameters: {'iterations': 1230, 'depth': 6, 'learning_rate': 0.0540169286925386, 'l2_leaf_reg': 0.3224885448071705, 'random_strength': 1.8556473239436837, 'bagging_temperature': 7.964382273601593, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 13, 'max_ctr_complexity': 2, 'one_hot_max_size': 195}. Best is trial 6 with value: 0.8027602070155262.
[I 2025-03-18 01:02:11,776] Trial 10 finished with value: 0.8085106382978723 and parameters: {'iterations': 1227, 'depth': 8, 'learning_rate': 0.06450118308764397, 'l2_leaf_reg': 7.580740427063586, 'random_strength': 0.46455210223216953, 'bagging_temperature': 9.312321878296636, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 14, 'max_ctr_complexity': 1, 'one_hot_max_size': 187}. Best is trial 10 with value: 0.8085106382978723.
[I 2025-03-18 01:02:11,872] Trial 0 finished with value: 0.8108108108108109 and parameters: {'iterations': 1484, 'depth': 7, 'learn

Best CatBoost parameters: {'iterations': 957, 'depth': 7, 'learning_rate': 0.022341665880888803, 'l2_leaf_reg': 3.535997573937274, 'random_strength': 0.29020788462819636, 'bagging_temperature': 1.0553916785367714, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 19, 'max_ctr_complexity': 1, 'one_hot_max_size': 242}
0:	learn: 0.6774262	test: 0.6778540	best: 0.6778540 (0)	total: 8.18ms	remaining: 7.82s
100:	learn: 0.3341943	test: 0.3967874	best: 0.3967874 (100)	total: 730ms	remaining: 6.18s
200:	learn: 0.2890636	test: 0.3811185	best: 0.3811185 (200)	total: 1.3s	remaining: 4.88s
300:	learn: 0.2608431	test: 0.3764918	best: 0.3764918 (300)	total: 1.83s	remaining: 3.98s
400:	learn: 0.2381301	test: 0.3749353	best: 0.3748907 (392)	total: 2.34s	remaining: 3.25s


...

[I 2025-03-18 01:19:12,492] A new study created in memory with name: no-name-703ba129-1085-48fa-9692-c8cf9ab084a0
[I 2025-03-18 01:19:12,571] Trial 8 finished with value: 0.8142610695802185 and parameters: {'model_type': 'logistic', 'C': 0.01245626328414047, 'solver': 'liblinear'}. Best is trial 8 with value: 0.8142610695802185.
[I 2025-03-18 01:19:12,581] Trial 1 finished with value: 0.8159861989649224 and parameters: {'model_type': 'logistic', 'C': 0.4070434730242739, 'solver': 'liblinear'}. Best is trial 1 with value: 0.8159861989649224.
[I 2025-03-18 01:19:12,630] Trial 11 finished with value: 0.8148361127084531 and parameters: {'model_type': 'logistic', 'C': 0.07462122377606112, 'solver': 'liblinear'}. Best is trial 1 with value: 0.8159861989649224.
[I 2025-03-18 01:19:12,636] Trial 10 finished with value: 0.8188614146060954 and parameters: {'model_type': 'logistic', 'C': 0.6723159356227854, 'solver': 'saga'}. Best is trial 10 with value: 0.8188614146060954.
[I 2025-03-18 01:19:12


==== Tuning Meta-Learner (11 parallel jobs) ====


[I 2025-03-18 01:19:12,972] Trial 2 finished with value: 0.8136860264519838 and parameters: {'model_type': 'xgboost', 'n_estimators': 61, 'max_depth': 6, 'learning_rate': 0.12377022353648637, 'subsample': 0.6526063721835276, 'colsample_bytree': 0.8389035216867546}. Best is trial 10 with value: 0.8188614146060954.
[I 2025-03-18 01:19:13,107] Trial 5 finished with value: 0.8182863714778609 and parameters: {'model_type': 'catboost', 'iterations': 180, 'depth': 6, 'learning_rate': 0.04381023113571828, 'l2_leaf_reg': 0.29269193761965595}. Best is trial 10 with value: 0.8188614146060954.
[I 2025-03-18 01:19:13,128] Trial 14 finished with value: 0.8102357676825762 and parameters: {'model_type': 'xgboost', 'n_estimators': 51, 'max_depth': 6, 'learning_rate': 0.1297906278214695, 'subsample': 0.6144977654353476, 'colsample_bytree': 0.6017983891479853}. Best is trial 10 with value: 0.8188614146060954.
[I 2025-03-18 01:19:13,206] Trial 19 finished with value: 0.8154111558366878 and parameters: {'m

Best meta-learner parameters: {'model_type': 'catboost', 'iterations': 478, 'depth': 3, 'learning_rate': 0.01909995100099854, 'l2_leaf_reg': 7.752355552270788}
Best threshold: 0.5051
Best meta-learner accuracy: 0.8223
Best meta-learner validation accuracy: 0.8223

===== FINAL PREDICTION =====
Optimized submission file created: optimized_submission.csv


{'ensemble': {'base_models': {'catboost': <catboost.core.CatBoostClassifier at 0x33a64be80>,
   'xgboost': XGBClassifier(base_score=None, booster=None, callbacks=None,
                 colsample_bylevel=None, colsample_bynode=None,
                 colsample_bytree=0.8807883364897827, device=None,
                 early_stopping_rounds=None, enable_categorical=True,
                 eval_metric=None, feature_types=None, gamma=0.38217207728815694,
                 grow_policy=None, importance_type=None,
                 interaction_constraints=None, learning_rate=0.04142242715327516,
                 max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
                 max_delta_step=None, max_depth=6, max_leaves=None,
                 min_child_weight=5, missing=nan, monotone_constraints=None,
                 multi_strategy=None, n_estimators=821, n_jobs=11,
                 num_parallel_tree=None, random_state=42, ...),
   'random_forest': RandomForestClassifier(max_depth=26